In [ ]:
import pandas as pd
import json

In [ ]:
with open('data/Melbourne/processed/crash/crashes.json') as f:
    jsonFile = json.load(f)

In [ ]:
jsonFile['T20060000010']

In [ ]:
jsonFile.

In [ ]:
for i in jsonFile:
    record = jsonFile[i]
    record['ACCIDENT_NO']=i
    print(record)

In [ ]:
record[['LAT', 'LON']]

In [ ]:
import argparse
import osmnx as ox
import fiona
import shutil
import os
import re
import csv
import geojson
import json
import requests
import yaml
import sys
from shapely.geometry import mapping, Polygon, LineString, LinearRing, Point

MAP_FP = None
PROCESSED_FP = None
PROCESSED_CRASHES_FP = None
CURR_FP = 'C:/Users/Daniel/Documents/ML/Transurban/src/data'
sys.path.append(CURR_FP)

import util

In [ ]:
config_file = '../config/Melbourne.yml'
data_dir = '../../data/Melbourne'

# Define file paths
MAP_FP = os.path.join(data_dir, 'processed/maps')
DOC_FP = os.path.join(data_dir, 'docs')
PROCESSED_FP = os.path.join(data_dir, 'processed')
PROCESSED_CRASHES_FP = os.path.join(PROCESSED_FP, 'crash')

In [ ]:
def find_osm_polygon(city):
    """Interrogate the OSM nominatim API for a city polygon.

    Nominatim may not always return city matches in the most intuitive order,
    so results need to be searched for a compatible polygon. The index of the
    polygon is required for proper use of osmnx.graph_from_place(). Some cities
    do not have a polygon at all, in which case they defer to using
    graph_from_point() with city lat & lng.

    Args:
        city (str): city to search for
    Returns:
        int: index of polygon+1 (becomes the correct 'which_result' value)
        None: if no polygon found
    """

    # Old Request that I couldn't get working for greater melbourne region
    # search_params = {'format': 'json', 'limit': 5, 'dedupe': 0, 'polygon_geojson': 1, 'city': city}
    # url = 'https://nominatim.openstreetmap.org/search'
    # response = requests.get(url, params=search_params)

    response = requests.get('https://nominatim.openstreetmap.org/search.php?q=greater+melbourne&polygon_geojson=1&format=json')
    for index, match in enumerate(response.json()):
        # To be used by graph_from_place needs to be a Polygon or MultiPolygon
        if (match['geojson']['type'] in ['Polygon', 'MultiPolygon']):
            return index + 1, match['geojson']

    return None, None

In [ ]:
polygon_pos, polygon = find_osm_polygon('swag')

In [ ]:
polygon = Polygon(polygon['coordinates'][0])

In [ ]:
G1 = ox.graph_from_polygon(polygon, network_type='drive', simplify=False)

In [ ]:
ox.plot.plot_graph(G1)

In [ ]:
G1

In [ ]:
G = ox.simplify_graph(G1)

In [ ]:
G

In [ ]:
ox.plot.plot_graph(G)

In [ ]:
streets_per_node = ox.count_streets_per_node(G)
for node, count in list(streets_per_node.items()):
    if count <= 1:
        G.nodes()[node]['dead_end'] = True

In [ ]:
G.nodes()[]

In [ ]:
ox.save_graph_shapefile(G1, filename='all_nodes', folder=MAP_FP)
ox.save_graph_shapefile(G, filename='temp', folder=MAP_FP)

In [ ]:
import argparse
import osmnx as ox
import fiona
import shutil
import os
import re
import csv
import geojson
import json
import requests
import yaml
import sys
from shapely.geometry import mapping, Polygon, LineString, LinearRing

MAP_FP = None
PROCESSED_FP = None
PROCESSED_CRASHES_FP = None
CURR_FP = 'C:/Users/Daniel/Documents/ML/Transurban'
sys.path.append(os.path.join(CURR_FP, 'src/data'))

import util

config = 'src/config/Melbourne.yml'
dataDir = 'data/Melbourne'
verbose = True
forceUpdate = False

MAP_FP = os.path.join(dataDir, 'processed/maps')
DOC_FP = os.path.join(dataDir, 'docs')
PROCESSED_FP = os.path.join(dataDir, 'processed')
PROCESSED_CRASHES_FP = os.path.join(PROCESSED_FP, 'crash')

In [ ]:
# Create maps [osm_nodes.shp, osm_ways.shp, osm_elements.geojson]
if not os.path.exists(os.path.join(MAP_FP, 'osm_ways.shp')) or forceUpdate:
    print('Generating map from open street map...')

In [ ]:
# CLEAN AND WRITE

ways_file = os.path.join(MAP_FP, 'osm_ways.shp')
nodes_file =os.path.join(MAP_FP, 'osm_nodes.shp')
result_file = os.path.join(MAP_FP, 'osm_elements.geojson')
DOC_FP = DOC_FP
verbose = verbose

In [ ]:
# CLEAN WAYS

orig_file = ways_file
DOC_FP = DOC_FP
verbose = verbose

In [ ]:
way_lines = fiona.open(orig_file)
highway_keys = {}
results = []

In [ ]:
for way_line in way_lines:

    speed = get_speed(way_line['properties']['maxspeed']) \
        if 'maxspeed' in list(way_line['properties']) else 0
    width = get_width(way_line['properties']['width']) \
        if 'width' in list(way_line['properties']) else 0

    lanes = way_line['properties']['lanes']
    if lanes:
        lanes = max([int(x) for x in re.findall('\d', lanes)])
    else:
        lanes = 0

    # Need to have an int highway field
    if way_line['properties']['highway'] not in list(highway_keys.keys()):
        highway_keys[way_line['properties']['highway']] = len(highway_keys)

    # Width per lane
    width_per_lane = 0
    if lanes and width:
        width_per_lane = round(width / lanes)

    # Use oneway
    oneway = 0
    if way_line['properties']['oneway'] == 'True':
        oneway = 1

    way_line['properties'].update({
        'width': width,
        'lanes': int(lanes),
        'hwy_type': highway_keys[way_line['properties']['highway']],
        'osm_speed': speed,
        'signal': 0,
        'oneway': oneway,
        'width_per_lane': width_per_lane
    })
    results.append(way_line)

In [ ]:
# GET SPEED
print(len(way_lines))
i = 0
for way_line in way_lines:
    if i in range(30):
        print('properties are', list(way_line['properties']))
        speed = way_line['properties']['maxspeed']
        print('speed is', speed)
        width = way_line['properties']['width']
        print('width is', width)


        lanes = way_line['properties']['lanes']
        if lanes:
            lanes = max([int(x) for x in re.findall('\d', lanes)])
        else:
            lanes = 0

        # Get one-way for each way
        oneway = 0
        if way_line['properties']['oneway'] == 'True':
            oneway = 1

        # Derive highway mapping
        if way_line['properties']['highway'] not in list(highway_keys.keys()):
            highway_keys[way_line['properties']['highway']] = len(highway_keys)

        # Derive width per lane
        width_per_lane = 0
        if lanes and width:
            width_per_lane = round(width / lanes)

        # Update properties with newly found variables
        # Note we include 'signal':0, but will properly update this varible later in write_geojson
        way_line['properties'].update({
            'width': width,
            'lanes': int(lanes),
            'hwy_type': highway_keys[way_line['properties']['highway']],
            'osm_speed': speed,
            'signal': 0, 
            'oneway': oneway,
            'width_per_lane': width_per_lane
        })
        
        print(way_line['properties'])
        
    i += 1
    
print(speed)


In [ ]:
if verbose:
    print('Within osm_create_maps.clean_ways with the following variables')
    print('orig file:', orig_file)

way_lines = fiona.open(orig_file)

highway_keys = {}
results = []
for way_line in way_lines:

    # Get speed for each way. Assign 0 if not found.
    speed = get_speed(way_line['properties']['maxspeed']) if 'maxspeed' in list(way_line['properties']) else 0

    # Get width for each way. Assign 0 if not found.
    width = get_width(way_line['properties']['width']) if 'width' in list(way_line['properties']) else 0

    # Get lanes for each way
    lanes = way_line['properties']['lanes']
    if lanes:
        lanes = max([int(x) for x in re.findall('\d', lanes)])
    else:
        lanes = 0

    # Get one-way for each way
    oneway = 0
    if way_line['properties']['oneway'] == 'True':
        oneway = 1

    # Derive highway mapping
    if way_line['properties']['highway'] not in list(highway_keys.keys()):
        highway_keys[way_line['properties']['highway']] = len(highway_keys)

    # Derive width per lane
    width_per_lane = 0
    if lanes and width:
        width_per_lane = round(width / lanes)

    # Update properties with newly found variables
    # Note we include 'signal':0, but will properly update this varible later in write_geojson
    way_line['properties'].update({
        'width': width,
        'lanes': int(lanes),
        'hwy_type': highway_keys[way_line['properties']['highway']],
        'osm_speed': speed,
        'signal': 0,
        'oneway': oneway,
        'width_per_lane': width_per_lane
    })

    results.append(way_line)

In [ ]:
def get_speed(speed):
    if speed:
        speeds = [int(x) for x in re.findall('\d+', speed)]
        if speeds:
            return max(speeds)
    return 0

In [ ]:
def get_width(width):
    """
    Parse the width from the openstreetmap width property field
    Args:
        width - a string
    Returns:
        width - an int
    """

    # This indicates two segments combined together.
    # For now, we just skip combined segments with different widths
    if not width or ';' in width or '[' in width:
        width = 0
    else:
        # Sometimes there's bad (non-numeric) width
        # so remove anything that isn't a number or .
        # Skip those that don't have some number in them
        width = re.sub('[^0-9\.]+', '', width)
        if width:
            width = round(float(width))
        else:
            width = 0
    return width

In [ ]:
nodes = fiona.open(nodes_file)
cleaned_ways = results
ways = results

In [ ]:
cleaned_ways[5]

In [ ]:
print(ways[0])

In [ ]:
a = 'hi, drew'
b = a.split(', ')
print(b)
"/".join(b)

In [ ]:
print(node_info)

In [ ]:
node_info = {}
for way in ways:

    # There are some collector roads and others that don't have names. Skip these.
    if way['properties']['name']:
        # While we are still merging segments with different names,
        # just use both roads. This should be revisited
        # e.g. [hi, drew] becomes 'hi/drew'
        if '[' in way['properties']['name']:
            way['properties']['name'] = re.sub(r'[^\s\w,]|_', '', way['properties']['name'])
            way['properties']['name'] = "/".join(way['properties']['name'].split(', '))


        if way['properties']['from'] not in node_info.keys():
            node_info[way['properties']['from']] = []
        node_info[way['properties']['from']].append(
            way['properties']['name'])

        if way['properties']['to'] not in node_info.keys():
            node_info[way['properties']['to']] = []
        node_info[way['properties']['to']].append(
            way['properties']['name'])

    ident = str(way['properties']['osmid']) + '-' \
        + str(way['properties']['from']) + '-' \
        + str(way['properties']['to'])
    way['properties']['segment_id'] = ident





In [ ]:
nodes_with_streets = []
for node in nodes:
    if node['properties']['osmid'] in node_info:
        print(node['properties']['osmid'])
        node['properties']['streets'] = ', '.join(
            set(node_info[node['properties']['osmid']]))
    else:
        node['properties']['streets'] = ''
    nodes_with_streets.append(node)

In [ ]:
nodes = nodes_with_streets
cleaned_ways = ways

In [ ]:
way_results = cleaned_ways
node_results = nodes
outfp = result_file

In [ ]:
feats = way_results

In [ ]:
len(feats)
feats

In [ ]:
for node in node_results:
    if not node['properties']['dead_end']:
        node['properties']['intersection'] = 1
    if node['properties']['highway'] == 'traffic_signals':
        node['properties']['signal'] = 1
    feats.append(geojson.Feature(
        geometry=geojson.Point(node['geometry']['coordinates']),
        properties=node['properties'])
    )

In [ ]:
node_results

In [ ]:
node_results

In [ ]:
len(feats)

In [ ]:
i = 0
j = 0
for feat in feats:

    if i == 0 and feat['geometry']['type'] == 'Point':
        print(feat)
        i+=1
    if j < 5 and feat['geometry']['type'] != 'Point':
        print(feat)
        j+=1
    if i > 0 and j > 5:
        break

In [ ]:
all_nodes_file =  os.path.join(MAP_FP, 'all_nodes', 'nodes', 'nodes.shp')

In [ ]:
all_node_results = fiona.open(all_nodes_file)
features = []

# Go through the rest of the nodes, and add any of them that have
# (hardcoded) open street map features that we care about
# For the moment, all_nodes only contains street nodes, so we'll
# only look at signals and crosswalks
for node in all_node_results:
    if node['properties']['highway'] == 'crossing':
        features.append(geojson.Feature(
            geometry=geojson.Point(node['geometry']['coordinates']),
            id=node['properties']['osmid'],
            properties={'feature': 'crosswalk'},
        ))

    elif node['properties']['highway'] == 'traffic_signals':
        features.append(geojson.Feature(
            geometry=geojson.Point(node['geometry']['coordinates']),
            id=node['properties']['osmid'],
            properties={'feature': 'signal'},
        ))



In [ ]:
features

In [ ]:
dataDir = 'C:/Users/Daniel/Documents/ML/Transurban/data/Melbourne'

In [ ]:

DATA_FP = dataDir
PROCESSED_DATA_FP = os.path.join(dataDir, 'processed')
MAP_FP = os.path.join(dataDir, 'processed/maps')


print("Within create_segments.py")

elements = os.path.join(MAP_FP, 'osm_elements.geojson')


In [ ]:
roads_shp_path = elements
mapfp = MAP_FP

In [ ]:
def reproject_records(records, inproj='epsg:4326', outproj='epsg:3857'):
    """
    Reprojects a set of records from one projection to another
    Records can either be points, line strings, or multiline strings
    Args:
        records - list of records to reproject
        inproj - defaults to 4326
        outproj - defaults to 3857
    Returns:
        list of reprojected records
    """
    results = []
    inproj = pyproj.Proj(init=inproj)
    outproj = pyproj.Proj(init=outproj)

    for record in records:

        coords = record['geometry']['coordinates']
        if record['geometry']['type'] == 'Point':
            re_point = pyproj.transform(inproj, outproj, coords[0], coords[1])
            point = Point(re_point)
            results.append({'geometry': point,
                            'properties': record['properties']})
        elif record['geometry']['type'] == 'MultiLineString':
            new_coords = []
            for segment in coords:
                new_segment = []
                for coord in segment:
                    new_segment.append(pyproj.transform(
                        inproj, outproj, coord[0], coord[1]))
                new_coords.append(new_segment)

            results.append({'geometry': MultiLineString(new_coords),
                            'properties': record['properties']})
        elif record['geometry']['type'] == 'LineString':
            new_coords = []
            for coord in coords:
                new_coords.append(
                    pyproj.transform(inproj, outproj, coord[0], coord[1])
                )
            results.append({'geometry': LineString(new_coords),
                            'properties': record['properties']})

    return results

In [ ]:
def get_roads_and_inters(filename):
    """
    Pull the roads and the intersections from a geojson file
    Typically this will read from the standardized osm_elements.geojson.
    Since that file includes dead ends, these will also be stripped.
    Everything will also be reprojected into 3857 projection
    Args:
        filename - geojson file of linestrings and points
    Returns:
        roads, intersections
    """

    # Read in osm_elements.geojson and reproject data to 3857
    data = fiona.open(filename)
    data = reproject_records([x for x in data])

    # Find all the roads by creating segments from those entries in osm_elements that have type LineString [i.e. are roads]
    roads = [seg(x['geometry'], x['properties']) for x in data if x['geometry'].type == 'LineString']

    # Get the intersection list by excluding anything that's not labeled as an intersection
    inters = [x for x in data if x['geometry'].type == 'Point' and 'intersection' in list(x['properties'].keys()) and x['properties']['intersection']]

    # Return roads [features that have type = 'LineString'] and intersections [features that have 'intersection' = True]
    return roads, inters

In [ ]:
import rtree
import json
import copy
from shapely.ops import unary_union
from collections import defaultdict
from . import util
import argparse
import os
import geojson
import re
from shapely.geometry import MultiLineString, LineString

import util
from segment import Segment as seg

In [ ]:
import fiona
import pyproj
import csv
import rtree
import geocoder
from time import sleep
from shapely.geometry import Point, shape, mapping, MultiLineString, LineString
from matplotlib import pyplot
import os
from os.path import exists as path_exists
import json
import sys
from dateutil.parser import parse
import datetime
import geojson

In [ ]:
filename = roads_shp_path

In [ ]:
# Read in osm_elements.geojson and reproject data to 3857
data = fiona.open(filename)
data = reproject_records([x for x in data])

In [ ]:
# Find all the roads by creating segments from those entries in osm_elements that have type LineString [i.e. are roads]
roads = [seg(x['geometry'], x['properties']) for x in data if x['geometry'].type == 'LineString']

# Get the intersection list by excluding anything that's not labeled as an intersection
inters = [x for x in data if x['geometry'].type == 'Point' and 'intersection' in list(x['properties'].keys()) and x['properties']['intersection']]


In [ ]:
    for i, road in enumerate(roads):
        road.properties['orig_id'] = int(str(99) + str(i))

In [ ]:
i=0
for road in roads:
     if i==100:
            print(road.properties['orig_id'])
     i+=1

In [ ]:
intersections = inters
intersection_buffer_units = 20
debug = True

In [ ]:
buffered_intersections = [intersection['geometry'].buffer(20) for intersection in intersections]

In [ ]:
len(buffered_intersections)

In [ ]:
type(buffered_intersections)

In [ ]:
util.output_from_shapes(
    [(x, {}) for x in buffered_intersections],
    os.path.join(MAP_FP, 'int_buffers.geojson')
)

In [ ]:
i = 0
for intersection in intersections:
    if i == 0:
        print(intersection)
    i+=1
    
i = 0
for buff_intersection in buffered_intersections:
    if i == 0:
        print(buff_intersection)
        buff_intersection
    i+=1

In [ ]:
a = 1
print(a)

In [ ]:
buffered_intersections = unary_union(buffered_intersections)

In [ ]:
len(buffered_intersections)

In [ ]:
inter_index = rtree.index.Index()
for idx, inter_point in enumerate(intersections):
    inter_index.insert(idx, inter_point['geometry'].bounds)


In [ ]:
results = []
i=0
for buff in buffered_intersections:
    matches = []
    if i ==0:
        print(buff.bounds)
        print(inter_index.intersection(buff.bounds))

    for idx in inter_index.intersection(buff.bounds):
        if i==0:
            print(idx)
            print(intersections[idx]['geometry'])
        if intersections[idx]['geometry'].within(buff):
            matches.append(intersections[idx]['geometry'])
            
    if i == 0:
        print(buff)
        print(matches)
    results.append([buff, matches])
    i+=1

In [ ]:
int_buffers = results
int_buffers

In [ ]:
def get_connections(points, segments):
    """
    Gets intersections by looking at the connections between points
    and segments that fall within an intersection buffer
    Args:
        points - a list of points
        segments - a list of segment objects
    Returns:
        A list of tuples for each intersection.
        Each tuple contains a set of segment objects
        and the buffer of the unary_union of the segment objects
        with a little bit of padding, because of a slight precision error
        in shapely operations
    """

    # Create a dict with each intersection point's coords as key
    # The values are the point itself and an empty list that will
    # store all the linestrings with a connection to the point
    inters = []
    for p in points:
        inters.append([p, []])

    # Get a starting list of all lines that touch any of the intersection points
    # i.e. any line in segments that is within 0.0001m of an intersection point will be
    # added to the intersection
    for line in segments:
        for i, (curr_shape, _) in enumerate(inters):
            if line.geometry.distance(curr_shape) < .0001:
                inters[i][1].append(line)
                inters[i][0] = unary_union([inters[i][0], line.geometry])

    # Merge connected components
    resulting_inters = []
    connected_lines = []
    while inters:
        curr = inters.pop(0)
        if inters:
            connected = [x[1] for x in inters if x[0].intersects(curr[0])]

            if connected:
                connected_lines = set(curr[1] + [x for y in connected for x in y])
            else:
                connected_lines = set(curr[1])
        else:
            connected_lines = set(curr[1])
        
        inters = [x for x in inters if not x[0].intersects(curr[0])]

        resulting_inters.append(
            (connected_lines, unary_union([x.geometry for x in connected_lines]).buffer(.001)))

    return resulting_inters

In [ ]:
def find_non_ints(roads, int_buffers):
    """
    Find the segments that aren't intersections
    Args:
        roads - a list of tuples of shapely shape and dict of segment info
        int_buffers - a list of polygons that buffer intersections
    Returns:
        tuple consisting of:
            non_int_lines - list in same format as input roads, just a subset
                each element in the list is a tuple of LineString or
                MultiLineString and dict of properties
            inter_segments - dict of lists with keys data and lines
                each element in the lines list is one of the lines
                overlapping the intersection buffer, and each element
                each element in the data list is a dict of properties
                corresponding to the lines
    """

    # Create index for quick lookup
    print("creating rindex")

    road_lines_index = rtree.index.Index()
    buffered_lines = []
    for idx, road in enumerate(roads):
        b = road.geometry.buffer(20)
        buffered_lines.append((b, road))
        road_lines_index.insert(idx, b.bounds)

    inter_segments = {'lines': defaultdict(list), 'data': defaultdict(list)}
    roads_with_int_segments = {}
    count = 0
    print("Generating intersection segments")
    for i, int_buffer in enumerate(int_buffers):
        util.track(i, 1000, len(int_buffers))
        match_segments = []
        matched_roads = []
        for idx in road_lines_index.intersection(int_buffer[0].bounds):
            road = roads[idx]
            match_segments.append(Segment(road.geometry.intersection(
                int_buffer[0]), road.properties))
            matched_roads.append(road)

        # Returns a list of tuples for each intersection
        # It connects road segments to intersections, and also combines close intersections.
        # Each tuple contains (set of segment objects, buffer(unary_union(segment_objects)))
        int_segments = get_connections(int_buffer[1], match_segments)

        # Each road_with_int is a road segment and a list of lists of segments
        # representing the intersections
        # to-do: turn these into intersection objects
        for r in matched_roads:
            if r.properties['id'] not in roads_with_int_segments:
                roads_with_int_segments[r.properties['id']] = []
            roads_with_int_segments[r.properties['id']] += int_segments

        for int_segment in int_segments:
            inter_segments['lines'][count] = [
                x.geometry for x in int_segment[0]]
            inter_segments['data'][count] = [
                x.properties for x in int_segment[0]]
            count += 1
    non_int_lines = []
    print("Generating non-intersection segments")
    for i, road in enumerate(roads):
        util.track(i, 1000, len(roads))
        # If there's no overlap between the road segment and any intersections
        if road.properties['id'] not in roads_with_int_segments:
            non_int_lines.append(geojson.Feature(
                geometry=geojson.LineString([x for x in road.geometry.coords]),
                properties=road.properties
            ))
        else:

            # Check against each separate intersection
            road_info = roads_with_int_segments[road.properties['id']]

            diff = road.geometry
            for inter in road_info:
                buffered_int = inter[1]
                diff = diff.difference(buffered_int)
            if 'LineString' == diff.type:
                non_int_lines.append(geojson.Feature(
                    geometry=geojson.LineString([x for x in diff.coords]),
                    properties=road.properties)
                )
            elif 'MultiLineString' == diff.type:
                coords = []
                for l in diff:
                    for coord in l.coords:
                        coords.append(coord)
                non_int_lines.append(geojson.Feature(
                    geometry=geojson.LineString(coords),
                    properties=road.properties)
                )
            else:
                # There may be no sections of the segment that fall outside
                # of an intersection, in which case it's skipped
                if len(diff) == 0:
                    continue
                print("{} found, skipping".format(diff.type))

    return non_int_lines, inter_segments


In [ ]:
non_int_lines, inter_segments = find_non_ints(roads, int_buffers)

In [ ]:
non_int_lines[0]

In [ ]:
inter_segments

In [ ]:
from create_segments import get_non_intersection_name
from create_segments import get_intersection_name

In [ ]:
# Allow intersections that don't have OSM IDs
# This will happen when we generate alternate maps from city data
# Though they won't have display names, we only use them to map to OSM segments
inters_by_id = {
    x['properties']['osmid'] if 'osmid' in x['properties'] else '0':
    x['properties']['streets'] if 'streets' in x['properties'] else None
    for x in inters
}

# For road lines that do not overlap with intersection, rejig some internal variables.
# Give them a display_name and append the result to non_int_w_ids
non_int_w_ids = []
for i, l in enumerate(non_int_lines):
    value = copy.deepcopy(l)
    value['type'] = 'Feature'
    value['properties']['id'] = '00' + str(i)
    value['properties']['inter'] = 0
    value['properties']['display_name'] = get_non_intersection_name(l, inters_by_id)
    non_int_w_ids.append(value)

    x, y = util.get_center_point(value)
    x, y = util.reproject([[x, y]], inproj='epsg:3857', outproj='epsg:4326')[0]['coordinates']
    value['properties']['center_y'] = y
    value['properties']['center_x'] = x

In [ ]:
non_int_w_ids[0]

In [ ]:
print("extracted {} non-intersection segments".format(len(non_int_w_ids)))

# Planarize intersection segments, turning the list of LineStrings into a MultiLineString
union_inter = []
for idx, lines in list(inter_segments['lines'].items()):

    lines = unary_union(lines)
    coords = []

    # Fix some mislabelling of dead-end node as intersection due to issue with OSMX
    if type(lines) == LineString:
        lines = MultiLineString([lines.coords])
    for line in lines:
        coords += [[x for x in line.coords]]

    # Assign name to intersection
    name = get_intersection_name(inter_segments['data'][idx])

    # Add the number of segments coming into this intersection
    segment_data = []
    for segment in list(inter_segments['data'][idx]):
        segment['intersection_segments'] = len(inter_segments['data'][idx])
        segment_data.append(segment)

    properties = {
        'id': idx,
        'data': segment_data,
        'display_name': name
    }

    value = geojson.Feature(
        geometry=geojson.MultiLineString(coords),
        id=idx,
        properties=properties,
    )

    x, y = util.get_center_point(value)
    x, y = util.reproject([[x, y]], inproj='epsg:3857',
                          outproj='epsg:4326')[0]['coordinates']

    value['properties']['center_x'] = x
    value['properties']['center_y'] = y
    union_inter.append(value)

In [ ]:
non_inters, inters = non_int_w_ids, union_inter

In [ ]:
non_inters[0]

In [ ]:
inters[10]

In [ ]:
from util import read_records_from_geojson
non_inters = non_inters
inters = inters
jsonfile = os.path.join(DATA_FP, 'processed', 'points_joined.json')
feats_filename=os.path.join(MAP_FP, 'features.geojson')
additional_feats_filename=None
forceupdate=False

In [ ]:
import util

In [ ]:
feats_filename

In [ ]:
from record import Record

In [ ]:

def read_records_from_geojson(filename):
    """
    Reads appropriately formatted geojson file,
    converts latitude and longitude to projection 4326, and turns into
    a Record object
    Args:
        filename - geojson file
    Returns:
        A list of Records
    """

    records = []
    with open(filename) as f:
        items = geojson.load(f)
        for item in items['features']:
            properties = item['properties']
            properties['location'] = {
                'LAT': item['geometry']['coordinates'][1],
                'LON': item['geometry']['coordinates'][0]
            }
            properties['LAT'] = item['geometry']['coordinates'][1]
            properties['LON'] = item['geometry']['coordinates'][0]
            record = Record(properties)
            records.append(record)
    return records

In [ ]:
if forceupdate or not os.path.exists(jsonfile):
    features = []
    if feats_filename:
        features = read_records_from_geojson(feats_filename)
    if additional_feats_filename:
        features += read_records(additional_feats_filename, 'record')

    print('Snapping {} point-based features'.format(len(features)))
    seg, segments_index = util.index_segments(
        inters + non_inters
    )

    # Add ['properties']['near_id'] to features according to overlapping buffer zones and distances
    util.find_nearest(features, seg, segments_index, 20, type_record=True)

    # Write the new points with IDs to identify their appropriate segments to file.
    print("Output {} point-based features to {}".format(len(features), jsonfile))
    with open(jsonfile, 'w') as f:
        json.dump([r.properties for r in features], f)

else:
    features = util.read_records(jsonfile, None)
    print("Read {} point-based features from file".format(len(features)))

In [ ]:
non_inters, inters, mapfp, datafp = non_inters, inters, MAP_FP, PROCESSED_DATA_FP

In [ ]:
non_inters = util.prepare_geojson(non_inters)

with open(os.path.join(mapfp, 'non_inters_segments.geojson'), 'w') as outfile:
    geojson.dump(non_inters, outfile)

# Get just the properties for the intersections
inter_data = {str(x['properties']['id']): x['properties']['data'] for x in inters}

with open(os.path.join(datafp, 'inters_data.json'), 'w') as f:
    json.dump(inter_data, f)

# Store the individual intersections without properties, since QGIS appears
# to have trouble with dicts of dicts, and viewing maps can be helpful
int_w_ids = [{
    'geometry': x['geometry'],
    'properties': {
        'id': x['properties']['id'],
        'display_name': x['properties']['display_name'] if 'display_name' in x['properties'] else '',
        'center_x': x['properties']['center_x'] if 'center_x' in x['properties'] else '',
        'center_y': x['properties']['center_y'] if 'center_y' in x['properties'] else ''
    }
} for x in inters]

int_w_ids = util.prepare_geojson(int_w_ids)

with open(os.path.join(mapfp, 'inters_segments.geojson'), 'w') as outfile:
    geojson.dump(int_w_ids, outfile)

# Store the combined segments with all properties
segments = non_inters['features'] + int_w_ids['features']

with open(os.path.join(mapfp, 'inter_and_non_int.geojson'), 'w') as outfile:
    geojson.dump(geojson.FeatureCollection(segments), outfile)

In [ ]:
from record import Crash

In [ ]:
import util

In [ ]:
import osm_create_maps

In [ ]:
import json
import pandas as pd
import os
import argparse
import sys

CURR_FP = 'C:/Users/Daniel/Documents/ML/Transurban/src/data'
sys.path.append(CURR_FP)

from util import read_geojson, group_json_by_location, group_json_by_field


BASE_DIR = 'C:/Users/Daniel/Documents/ML/Transurban'
MAP_FP = os.path.join(BASE_DIR, 'data/processed/maps')
DATA_FP = os.path.join(BASE_DIR, 'data/processed')


In [ ]:
datadir = 'data/Melbourne'
config = 'src/config/Melbourne.yml'

In [ ]:
feats = ['AADT', 'SPEEDLIMIT', 'Struct_Cnd', 'Surface_Tp', 'F_F_Class']

In [ ]:
def aggregate_roads(feats, datadir):

    # read/aggregate crash/concerns
    crash = read_records(os.path.join(datadir, 'crash_joined.json'), 'dateOccurred', 'near_id')
    cr_con = pd.concat([crash], axis=1)
    cr_con.columns = ['crash']

    # if null for a certain week = 0 (no crash)
    cr_con.reset_index(inplace=True)
    cr_con = cr_con.fillna(0)
    # Make near_id string (for matching to segments)
    cr_con['near_id'] = cr_con['near_id'].astype('str')

    # combined road feature dataset parameters
    inters_fp = os.path.join(datadir, 'inters_data.json')
    non_inters_fp = os.path.join(datadir, 'maps', 'non_inters_segments.geojson')

    # create combined road feature dataset
    aggregated, adjacent = road_make(feats, inters_fp, non_inters_fp)
    print("road features being included: ", ', '.join(feats))

    # Add any concern types if applicable
    filename = os.path.join(datadir, 'concern_joined.json')
    if os.path.exists(filename):

        concerns = read_concerns(
            filename, 'near_id'
        )
        for concern_type, result in concerns:

            f = {concern_type: result}
            aggregated = aggregated.assign(**f)
            aggregated = aggregated.fillna(0)
    else:
        aggregated = aggregated.fillna(0)

    # All features as int
    aggregated = aggregated.apply(lambda x: x.astype('int'))

    return aggregated, adjacent, cr_con

In [ ]:
def read_records(fp, date_col, id_col, agg='week'):
    """ Read point data, output count by aggregation level
    agg : datepart for aggregation
    date_col : column name with date information
    id_col : column name with inter/non-inter id (for grouping)
    """

    with open(fp, 'r') as f:
        data = json.load(f)
    df = pd.DataFrame(data)

    df[date_col] = pd.to_datetime(df[date_col])
    print("total number of records in {}:{}".format(fp, len(df)))
    
    # get date according to iso calendar
    df['isodate'] = df[date_col].apply(lambda d: d.isocalendar())
    df['week'] = df['isodate'].apply(lambda x: x[1])
    df['year'] = df['isodate'].apply(lambda x: x[0])

    # aggregate
    print("aggregating by ", agg)
    df[agg] = df[date_col].apply(lambda x: getattr(x, agg))
    df_g = df.groupby([id_col, 'year', agg]).size()

    return(df_g)

In [ ]:
aggregated, adjacent, cr_con = aggregate_roads(feats, DATA_FP)

In [ ]:
import json

In [ ]:
filePath = 'C:/Users/Daniel/Documents/ML/Transurban/data/Melbourne/processed/crash_joined.json' 
with open(filePath) as f:
    data = json.loads(f.read())

In [ ]:
filePath2 = 'C:/Users/Daniel/Documents/ML/Transurban/data/Melbourne/processed/inters_data.json' 
with open(filePath2) as f:
    intersGeoData = json.loads(f.read())

In [ ]:
filePath3 = 'C:/Users/Daniel/Documents/ML/Transurban/data/Melbourne/processed/maps/inter_and_non_int.geojson' 
with open(filePath3) as f:
    combinedGeoData = json.loads(f.read())

In [ ]:
filePath4 = 'C:/Users/Daniel/Documents/ML/Transurban/data/Melbourne/processed/maps/non_inters_segments.geojson' 
with open(filePath4) as f:
    nonIntersGeoData = json.loads(f.read())

In [ ]:
for feat in combinedGeoData['features']:
    properties = feat['properties']
    
    if 'id' in properties:
        myID = properties['id']
    
    if 'orig_id' in properties:
        orig_id = properties['orig_id']

    if myID == '0056475':
        print('my ID match')
        print(feat)
    
    if orig_id == '9956475':
        print('orig ID match')
        print(feat)
        
    if orig_id == '37247':
        print('orig ID match')
        print(feat)

In [ ]:
data

In [ ]:
intersGeoData

In [ ]:
nonIntersGeoData

In [ ]:
import json
import pandas as pd
import os
import argparse

CURR_FP = 'C:/Users/Daniel/Documents/ML/Transurban/src/data'
sys.path.append(CURR_FP)

from util import read_geojson, group_json_by_location, group_json_by_field


In [ ]:
dataDir = 'C:/Users/Daniel/Documents/ML/Transurban/data/Melbourne'

In [ ]:
DATA_FP = os.path.join(dataDir, 'processed')
MAP_FP = os.path.join(DATA_FP, 'maps')

In [ ]:
feats = ['AADT', 'SPEEDLIMIT', 'Struct_Cnd', 'Surface_Tp', 'F_F_Class']
crashed_joined_fp = os.path.join(DATA_FP, 'crash_joined.json')
fp = crashed_joined_fp

In [ ]:
with open(fp, 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)

In [ ]:
date_col = 'ACCIDENT_DATE'
date_col_iso = "ACCIDENT_DATE_ISO"
df[date_col] = pd.to_datetime(df[date_col])
df[date_col_iso] = df[date_col].apply(lambda d: d.isocalendar())

In [ ]:
import numpy as np

In [ ]:
newDF = df.copy()

In [ ]:
newDF['near_id'].replace('', np.nan, inplace=True)
newDF.dropna(inplace=True)
newDF['near_id'] = newDF['near_id'].astype(str)
newDF.sort_index(inplace=True)
#newDF.sort_values('near_id', inplace=True)

In [ ]:
newDF.ACCIDENT_DATE[0]

In [ ]:
newDF.head()

In [ ]:
newDF['LAST_7_DAYS'] = ''
newDF['LAST_30_DAYS'] = ''
newDF['LAST_365_DAYS'] = ''
newDF['LAST_1825_DAYS'] = ''
newDF['LAST_3650_DAYS'] = ''

In [ ]:
newDF.sort_values('ACCIDENT_DATE', inplace=True)

In [ ]:
accidents_list = []
lenDF = len(newDF)
for i in range(0,len(newDF)):
    accidents_same_location = newDF[newDF.near_id == newDF.iloc[i].near_id]
    accidents_same_location_date = accidents_same_location.ACCIDENT_DATE
    
    current_accident_date = newDF.iloc[i].ACCIDENT_DATE
    past_7_days = current_accident_date - pd.to_timedelta("7day")
    past_30_days = current_accident_date - pd.to_timedelta("30day")
    past_365_days = current_accident_date - pd.to_timedelta("365day")
    past_1825_days = current_accident_date - pd.to_timedelta("1825day")
    past_3650_days = current_accident_date - pd.to_timedelta("3650day")
    
    idx_in_the_past = [accidents_same_location_date < current_accident_date]
    idx_last_7_days = [accidents_same_location_date > past_7_days]
    idx_last_30_days = [accidents_same_location_date > past_30_days]
    idx_last_365_days = [accidents_same_location_date > past_365_days]
    idx_last_1825_days = [accidents_same_location_date > past_1825_days]
    idx_last_3650_days = [accidents_same_location_date > past_3650_days]
    
    idx_7 = [x & y for (x,y) in zip(idx_in_the_past, idx_last_7_days)]
    idx_30 = [x & y for (x,y) in zip(idx_in_the_past, idx_last_30_days)]
    idx_365 = [x & y for (x,y) in zip(idx_in_the_past, idx_last_365_days)]
    idx_1825 = [x & y for (x,y) in zip(idx_in_the_past, idx_last_1825_days)]
    idx_3650 = [x & y for (x,y) in zip(idx_in_the_past, idx_last_3650_days)]
    
    accidents_last_7_days = sum(sum((idx_7)))
    accidents_last_30_days = sum(sum((idx_30)))
    accidents_last_365_days = sum(sum((idx_365)))
    accidents_last_1825_days = sum(sum((idx_1825)))
    accidents_last_3650_days = sum(sum((idx_3650)))

    accidents_list.append([accidents_last_7_days, accidents_last_30_days, accidents_last_365_days, accidents_last_1825_days, accidents_last_3650_days])
    
    if i % 1000 == 0:
        print(i / lenDF)

In [ ]:
import csv

with open("output.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(accidents_list)

In [ ]:
newDF.head()

In [ ]:
LAST_N_DAYS_LIST = ['LAST_7_DAYS', 'LAST_30_DAYS', 'LAST_365_DAYS', 'LAST_1825_DAYS', 'LAST_3650_DAYS']

In [ ]:
newDF[LAST_N_DAYS_LIST] = accidents_list

In [ ]:
newDF.tail()

In [ ]:
newDF.sort_values(['near_id', 'ACCIDENT_DATE'])[['ACCIDENT_DATE', 'near_id', 'LAST_7_DAYS', 'LAST_30_DAYS', 'LAST_365_DAYS', 'LAST_1825_DAYS', 'LAST_3650_DAYS']]

In [ ]:
df[date_col] = pd.to_datetime(df[date_col])
print("total number of records in {}:{}".format(fp, len(df)))

# get date according to iso calendar
df['ACCIDENT_DATE_ISO'] = df[date_col].apply(lambda d: d.isocalendar())

In [ ]:
df.head()

In [ ]:

# aggregate
print("aggregating by ", agg)
df[agg] = df[date_col].apply(lambda x: getattr(x, agg))
df_g = df.groupby([id_col, 'year', agg]).size()

In [ ]:
import json
import pandas as pd
import os
import argparse
import numpy as np
import csv
import sys

BASE_DIR= 'C:/Users/Daniel/Documents/ML/Transurban'
CURR_FP = 'C:/Users/Daniel/Documents/ML/Transurban/src/data'
MAP_FP = os.path.join(BASE_DIR, 'data/Melbourne/processed/maps')
DATA_FP = os.path.join(BASE_DIR, 'data/Melbourne/processed')
sys.path.append(CURR_FP)

sys.path.append(CURR_FP)

from util import read_geojson, group_json_by_location, group_json_by_field

In [ ]:
def read_records(data_fp, forceupdate=False):
    """ Read point data, output count by aggregation level
    agg : datepart for aggregation
    id_col : column name with inter/non-inter id (for grouping)
    """

    # Load in crash data
    with open(os.path.join(data_fp, 'crash_joined.json'), 'r') as f:
        data = json.load(f)
    df = pd.DataFrame(data)

    # Write date in pd friendly format
    # Get ISO date for easier date calculations
    date_col = 'ACCIDENT_DATE'
    date_col_iso = "ACCIDENT_DATE_ISO"
    df[date_col] = pd.to_datetime(df[date_col])
    df[date_col_iso] = df[date_col].apply(lambda d: d.isocalendar())

    # DF cleaning
    # First, drop accidents that don't have a near_id mapping
    # Turn 'near_id' to a string, as it may be mixed datatype here
    # Sort by accident date for data transparency
    df['near_id'].replace('', np.nan, inplace=True)
    df.dropna(inplace=True)
    df['near_id'] = df['near_id'].astype(str)
    df.sort_values('ACCIDENT_DATE', inplace=True)

    # Add columsn regarding historical crash counts to each event in crashes.csv
    # Currently 7 day, 30 day, 365 day, 1825 day and 3650 day time periods
    # This takes a long time to run, so try to load instead from csv file if already exists
    accident_count_fp = os.path.join(data_fp, 'crash/accident_count.csv')

    if forceupdate or not os.path.exists(accident_count_fp):
        print('In make_canon_dataset. About to append accident count by time period to crash data. This takes a long time and is unoptimised / not DRY. Fix in future.')
        accident_count = []
        lenDF = len(df)
        for i in range(0, len(df)):
            accident_same_location = df[df.near_id == df.iloc[i].near_id]
            accident_same_location_date = accident_same_location.ACCIDENT_DATE

            current_accident_date = df.iloc[i].ACCIDENT_DATE
            past_7_days = current_accident_date - pd.to_timedelta("7day")
            past_30_days = current_accident_date - pd.to_timedelta("30day")
            past_365_days = current_accident_date - pd.to_timedelta("365day")
            past_1825_days = current_accident_date - pd.to_timedelta("1825day")
            past_3650_days = current_accident_date - pd.to_timedelta("3650day")

            idx_in_the_past = [accident_same_location_date < current_accident_date]
            idx_last_7_days = [accident_same_location_date > past_7_days]
            idx_last_30_days = [accident_same_location_date > past_30_days]
            idx_last_365_days = [accident_same_location_date > past_365_days]
            idx_last_1825_days = [accident_same_location_date > past_1825_days]
            idx_last_3650_days = [accident_same_location_date > past_3650_days]

            idx_7 = [x & y for (x, y) in zip(idx_in_the_past, idx_last_7_days)]
            idx_30 = [x & y for (x, y) in zip(idx_in_the_past, idx_last_30_days)]
            idx_365 = [x & y for (x, y) in zip(idx_in_the_past, idx_last_365_days)]
            idx_1825 = [x & y for (x, y) in zip(idx_in_the_past, idx_last_1825_days)]
            idx_3650 = [x & y for (x, y) in zip(idx_in_the_past, idx_last_3650_days)]

            accident_last_7_days = sum(sum((idx_7)))
            accident_last_30_days = sum(sum((idx_30)))
            accident_last_365_days = sum(sum((idx_365)))
            accident_last_1825_days = sum(sum((idx_1825)))
            accident_last_3650_days = sum(sum((idx_3650)))

            accident_count.append([accident_last_7_days, accident_last_30_days, accident_last_365_days, accident_last_1825_days, accident_last_3650_days])

            if i % 1000 == 0:
                print('{}% done.'.format(i / lenDF * 100))

        with open("accident_count.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(accident_count)

    # If the file already exists then we can simple read it in.
    # Note the list comprehensioin is due to the strange saving behaviour of previous section, leaves empty rows
    else:
        with open(accident_count_fp, 'r') as f:
            print('Untested reading in of .csv')
            reader = csv.reader(f, delimiter=',')
            accident_count = list(list(values) for values in reader)
            accident_count = [x for x in accident_count if x]

    # Now append the new accident counts to the crashes df
    LAST_N_DAYS_LIST = ['LAST_7_DAYS', 'LAST_30_DAYS', 'LAST_365_DAYS', 'LAST_1825_DAYS', 'LAST_3650_DAYS']
    df[LAST_N_DAYS_LIST] = accident_count

    return df


In [ ]:
data_fp = DATA_FP

In [ ]:
# Load in crash data
with open(os.path.join(data_fp, 'crash_joined.json'), 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)

# Write date in pd friendly format
# Get ISO date for easier date calculations
date_col = 'ACCIDENT_DATE'
date_col_iso = "ACCIDENT_DATE_ISO"
df[date_col] = pd.to_datetime(df[date_col])
df[date_col_iso] = df[date_col].apply(lambda d: d.isocalendar())

# DF cleaning
# First, drop accidents that don't have a near_id mapping
# Turn 'near_id' to a string, as it may be mixed datatype here
# Sort by accident date for data transparency
df['near_id'].replace('', np.nan, inplace=True)
df.dropna(inplace=True)
df['near_id'] = df['near_id'].astype(str)
df.sort_values('ACCIDENT_DATE', inplace=True)

# Add columsn regarding historical crash counts to each event in crashes.csv
# Currently 7 day, 30 day, 365 day, 1825 day and 3650 day time periods
# This takes a long time to run, so try to load instead from csv file if already exists
accident_count_fp = os.path.join(data_fp, 'crash/accident_count.csv')


In [ ]:
accident_count_fp

In [ ]:
os.path.exists(accident_count_fp)

In [ ]:
        with open(accident_count_fp, 'r') as f:
            print('Untested reading in of .csv')
            reader = csv.reader(f, delimiter=',')
            accident_list = list(list(values) for values in reader)

In [ ]:
accident_list

In [ ]:
accident_list = [x for x in accident_list if x]

In [ ]:
list2


In [ ]:
df.head()

In [ ]:
with open(accident_count_fp, 'r') as f:
    print('Untested reading in of .csv')
    reader = csv.reader(f, delimiter=',')
    accident_count = list(list(values) for values in reader)
    accident_count = [x for x in accident_count if x]

# Now append the new accident counts to the crashes df
LAST_N_DAYS_LIST = ['LAST_7_DAYS', 'LAST_30_DAYS', 'LAST_365_DAYS', 'LAST_1825_DAYS', 'LAST_3650_DAYS']
df[LAST_N_DAYS_LIST] = accident_count

In [ ]:
LAST_N_DAYS_LIST = ['LAST_7_DAYS', 'LAST_30_DAYS', 'LAST_365_DAYS', 'LAST_1825_DAYS', 'LAST_3650_DAYS']
df[LAST_N_DAYS_LIST] = accident_list


In [ ]:
df['LAST_7_DAYS'] = ''
df['LAST_30_DAYS'] = ''
df['LAST_365_DAYS'] = ''
df['LAST_1825_DAYS'] = ''
df['LAST_3650_DAYS'] = ''
LAST_N_DAYS_LIST = ['LAST_7_DAYS', 'LAST_30_DAYS', 'LAST_365_DAYS', 'LAST_1825_DAYS', 'LAST_3650_DAYS']
df[LAST_N_DAYS_LIST] = accident_count


In [ ]:
df.tail()

In [ ]:
a = pd.concat([df], axis=1)

In [ ]:
a.columns = ['crash']

In [ ]:
inters_fp = os.path.join(datadir, 'inters_data.json')
non_inters_fp = os.path.join(datadir, 'maps', 'non_inters_segments.geojson')

In [ ]:
datadir = DATA_FP

In [ ]:

agg = 'max'

In [ ]:
df_index = []
df_records = []
print("reading ", inters_fp)


In [ ]:
with open(inters_fp, 'r') as f:
    inters = json.load(f)
    # Append each index to dataframe
    for idx, lines in inters.items():

        # Each intersection has more than one segment
        # Add each segment's properties to df_records
        df_records.extend(lines)
        df_index.extend([idx] * len(lines))


In [ ]:
df_index

In [ ]:
inters_df = pd.DataFrame(df_records, index=df_index)

In [ ]:
inters_df

In [ ]:
# Read in non_inters data
print("Reading in:", non_inters_fp)
non_inters = read_geojson(non_inters_fp)
non_inters_df = pd.DataFrame([x[1] for x in non_inters])
non_inters_df.set_index('id', inplace=True)

In [ ]:
i =0
for x in non_inters:
    if i == 0:
        print(x)
        print(x[0])
        print(x[1])
    i += 1

In [ ]:
combined = pd.concat([inters_df, non_inters_df])

In [90]:
inters_df.columns.sort_values()

Index(['access', 'area', 'bridge', 'dead_end', 'est_width', 'from', 'highway',
       'hwy_type', 'id', 'intersection', 'intersection_segments', 'junction',
       'key', 'lanes', 'length', 'maxspeed', 'name', 'oneway', 'orig_id',
       'osm_speed', 'osmid', 'ref', 'segment_id', 'service', 'signal',
       'streets', 'to', 'tunnel', 'width', 'width_per_lane'],
      dtype='object')

In [89]:
non_inters_df.columns.sort_values()

Index(['access', 'area', 'bridge', 'center_x', 'center_y', 'dead_end',
       'display_name', 'est_width', 'from', 'highway', 'hwy_type', 'inter',
       'intersection', 'junction', 'key', 'lanes', 'length', 'maxspeed',
       'name', 'oneway', 'orig_id', 'osm_speed', 'osmid', 'ref', 'segment_id',
       'service', 'signal', 'streets', 'to', 'tunnel', 'width',
       'width_per_lane'],
      dtype='object')

In [91]:
combined.columns.sort_values()

Index(['access', 'area', 'bridge', 'center_x', 'center_y', 'dead_end',
       'display_name', 'est_width', 'from', 'highway', 'hwy_type', 'id',
       'inter', 'intersection', 'intersection_segments', 'junction', 'key',
       'lanes', 'length', 'maxspeed', 'name', 'oneway', 'orig_id', 'osm_speed',
       'osmid', 'ref', 'segment_id', 'service', 'signal', 'streets', 'to',
       'tunnel', 'width', 'width_per_lane'],
      dtype='object')

In [93]:
pd.set_option('display.max_columns', 500)

In [94]:
combined.head()

,access,area,bridge,center_x,center_y,dead_end,display_name,est_width,from,highway,hwy_type,id,inter,intersection,intersection_segments,junction,key,lanes,length,maxspeed,name,oneway,orig_id,osm_speed,osmid,ref,segment_id,service,signal,streets,to,tunnel,width,width_per_lane
0,None,None,None,NaN,NaN,None,NaN,None,110895756,tertiary,6,216513,NaN,None,3.0,None,0,0,2502.4690000000005,None,Cape Schanck Road,0,99216513,0,5431724,None,5431724-110895756-1780088077,None,0,None,1780088077,None,0,0
0,None,None,None,NaN,NaN,None,NaN,None,110895756,tertiary,6,228349,NaN,None,2.0,roundabout,0,0,47.47400000000001,None,None,1,99228349,0,172237581,None,172237581-110895756-110895756,None,0,None,110895756,None,0,0
1,None,None,None,NaN,NaN,None,NaN,None,1368579751,residential,0,175149,NaN,None,2.0,None,0,0,70.999,None,Blowhole Track,0,99175149,0,122483327,None,122483327-1368579751-1368579751,None,0,None,1368579751,None,0,0
1,None,None,None,NaN,NaN,None,NaN,None,1368579751,residential,0,175148,NaN,None,3.0,None,0,0,416.32899999999995,None,Blowhole Track,0,99175148,0,122483327,None,122483327-1368579751-1368579746,None,0,None,1368579746,None,0,0
2,None,None,None,NaN,NaN,None,NaN,None,110837756,residential,0,222544,NaN,None,3.0,None,0,0,1485.1929999999998,None,Golf Links Road,1,99222544,0,161758042,None,161758042-110837756-110837788,None,0,None,110837788,None,0,0


In [95]:
combined.tail()

,access,area,bridge,center_x,center_y,dead_end,display_name,est_width,from,highway,hwy_type,id,inter,intersection,intersection_segments,junction,key,lanes,length,maxspeed,name,oneway,orig_id,osm_speed,osmid,ref,segment_id,service,signal,streets,to,tunnel,width,width_per_lane
00181284,None,None,None,145.012401,-37.669542,None,Ryder Street between Richards Street and Dicke...,None,2460519344,residential,0,NaN,0.0,None,NaN,None,0,0,86.072,50,Ryder Street,0,99248654,50,238199040,None,238199040-2460519344-2460519337,None,0,None,2460519337,None,0,0
00181285,None,None,None,144.914685,-37.693619,None,Eleanor Street between Langton Street/Eleanor ...,None,301989623,residential,0,NaN,0.0,None,NaN,None,0,1,114.15100000000001,None,Eleanor Street,1,99248655,0,27507426,None,27507426-301989623-301989596,None,0,None,301989596,None,0,0
00181286,None,None,yes,144.913984,-37.695004,None,Langton Street between Gervase Avenue and Elea...,None,301989748,residential,0,NaN,0.0,None,NaN,None,0,0,334.054,None,Langton Street,0,99248656,0,"[27507440, 27507444, 27507439]",None,"[27507440, 27507444, 27507439]-301989748-30198...",None,0,None,301989623,None,0,0
00181287,None,None,None,144.913874,-37.700883,None,Langton Street between Anselm Grove and Arunde...,None,301989773,residential,0,NaN,0.0,None,NaN,None,0,0,329.744,None,Langton Street,0,99248660,0,27507446,None,27507446-301989773-301989772,None,0,None,301989772,None,0,0
00181288,None,None,None,145.094574,-37.835564,None,Riversdale Road between Union Road and Verdun ...,None,1837365646,secondary,2,NaN,0.0,None,NaN,None,0,0,170.321,60,Riversdale Road,0,99248661,60,198141499,20,198141499-1837365646-1837365645,None,0,None,1837365645,None,0,0


In [134]:
feats_osm = ['display_name', 'name', 'hwy_type', 'inter', 'intersection_segments', 'junction', 'key', 'lanes', 'name', 'oneway', 'orig_id', 'osm_speed', 'segment_id', 'service', 'signal', 'streets', 'to', 'width', 'width_per_lane']

In [101]:
aggregated = getattr(combined[feats_osm].groupby(combined.index), agg)

In [105]:
aggregated_df = aggregated()

In [109]:
df.head()

,ACCIDENT_DATE,ACCIDENT_NO,ACCIDENT_TIME,ATMOSPH_COND,COMPLEX_INT_NO,COMPLEX_NODE,DAY_OF_WEEK,DEGREE_URBAN,HOUR,LAT,LIGHT_CONDITION,LON,NODE_ID,NODE_TYPE_INT,ROAD_GEOMETRY,SPEED_ZONE,SUBURB,near_id,ACCIDENT_DATE_ISO,LAST_7_DAYS,LAST_30_DAYS,LAST_365_DAYS,LAST_1825_DAYS,LAST_3650_DAYS
3203,2006-01-01,T20060011459,13.10.00,1,0,0,1,MELB_URBAN,13,-37.81530,1,145.26929,55650,0,2,70,MAROONDAH,54917,"(2005, 52, 7)",0,0,0,0,0
1353,2006-01-01,T20060005342,02.00.00,1,0,0,1,MELB_URBAN,02,-38.29991,3,145.17389,54769,0,2,70,MORNINGTON PENINSULA,3805,"(2005, 52, 7)",0,0,0,0,0
1375,2006-01-01,T20060005387,10.00.00,1,0,0,1,RURAL_VICTORIA,10,-38.25092,9,145.19081,35657,0,2,80,MORNINGTON PENINSULA,4116,"(2005, 52, 7)",0,0,0,0,0
1453,2006-01-01,T20060005600,12.35.00,1,0,0,1,RURAL_VICTORIA,12,-38.00289,1,145.52014,202068,1,5,80,CARDINIA,0060650,"(2005, 52, 7)",0,0,0,0,0
1454,2006-01-01,T20060005607,11.20.00,1,0,0,1,SMALL_CITIES,11,-38.07784,1,145.48576,55403,0,2,50,CARDINIA,12768,"(2005, 52, 7)",0,0,0,0,0


In [138]:
aggregated_df

,display_name,hwy_type,inter,intersection_segments,key,lanes,oneway,orig_id,osm_speed,segment_id,signal,streets,to,width,width_per_lane
0,NaN,6,NaN,3.0,0,0,1,99228349,0,5431724-110895756-1780088077,0,NaN,1780088077,0,0
000,Muntz Avenue between Pecham Street/Nene Avenue...,0,0.0,NaN,0,0,0,990,0,27507451-374572331-301989903,0,NaN,301989903,0,0
001,Augustine Terrace between Ridgeway Avenue and ...,0,0.0,NaN,0,0,0,991,0,202392466-374632361-301989903,0,NaN,301989903,0,0
0010,Ayr Street between South Street and Ailsa Street,0,0.0,NaN,0,0,0,9913,0,4917943-32211137-444596322,0,NaN,444596322,0,0
00100,Anderson Road from Durham Road,5,0.0,NaN,0,0,1,99163,60,198875958-309595876-1843397341,0,NaN,1843397341,0,0
001000,Marble Drive from Opal Street,0,0.0,NaN,0,0,0,991504,0,375616293-3789728768-4869595494,0,NaN,4869595494,0,0
0010000,Velra Avenue between North Road and Donal Street,0,0.0,NaN,0,0,0,9913917,0,23810526-258024347-258024346,0,NaN,258024346,0,0
00100000,Winjeel Court from Stores Court/Winjeel Court/...,0,0.0,NaN,0,0,0,99136535,0,41452835-290320909-290312737,0,NaN,290312737,0,0
00100001,Air Force Avenue from West Central Park Avenue,0,0.0,NaN,0,0,0,99136536,0,41454161-290320942-290312739,0,NaN,290312739,0,0
00100002,Stella Place from Maria Crescent,0,0.0,NaN,0,0,0,99136538,0,32032512-359518836-359518814,0,NaN,359518814,0,0


In [129]:
df.tail()

,ACCIDENT_DATE,ACCIDENT_NO,ACCIDENT_TIME,ATMOSPH_COND,COMPLEX_INT_NO,COMPLEX_NODE,DAY_OF_WEEK,DEGREE_URBAN,HOUR,LAT,LIGHT_CONDITION,LON,NODE_ID,NODE_TYPE_INT,ROAD_GEOMETRY,SPEED_ZONE,SUBURB,near_id,ACCIDENT_DATE_ISO,LAST_7_DAYS,LAST_30_DAYS,LAST_365_DAYS,LAST_1825_DAYS,LAST_3650_DAYS
171890,2018-12-10,T20180019126,08.45.00,1,0,0,5,MELB_URBAN,08,-37.81086,1,145.23477,337472,1,5,80,MAROONDAH,0046676,"(2018, 50, 1)",0,0,0,1,4
171894,2018-12-10,T20180019139,12.44.00,1,0,0,5,MELBOURNE_CBD,12,-37.81272,1,144.97216,335436,1,2,40,MELBOURNE,0013280,"(2018, 50, 1)",0,0,0,1,3
171893,2018-12-10,T20180019137,10.00.00,1,0,0,6,MELB_URBAN,10,-37.77496,1,144.97114,35987,0,1,40,MORELAND,65293,"(2018, 50, 1)",0,0,0,7,9
171888,2018-12-10,T20180019119,03.50.00,1,0,0,5,MELB_URBAN,03,-37.65250,3,145.02324,335426,1,5,60,WHITTLESEA,89224,"(2018, 50, 1)",0,0,0,0,1
171905,2018-12-10,T20180019164,15.15.00,1,9083,1,5,MELB_URBAN,15,-37.74359,1,145.04758,56886,0,2,60,BANYULE,73541,"(2018, 50, 1)",0,0,1,1,3


In [135]:
aggregated_df.loc['0013280']['display_name']

'Little Collins Street between Spring Street and Exhibition Street'

In [131]:
inters_df.loc['0013280']

KeyError: 'the label [0013280] is not in the [index]'

In [133]:
non_inters_df.loc['0013280']

access                                                         None
area                                                           None
bridge                                                         None
est_width                                                      None
from                                                     2279153639
highway                                                unclassified
junction                                                       None
key                                                               0
lanes                                                             0
length                                                      219.916
maxspeed                                                         40
name                                          Little Collins Street
oneway                                                            1
osmid                                                       4306021
ref                                             

In [144]:
merged = pd.merge(df, aggregated_df, left_on='near_id', right_index=True, how='left')

In [145]:
merged.shape

(123240, 39)

In [148]:
print("There are {} rows with no missing values.".format((merged.apply(lambda x: x.isnull().sum(), axis= 1)==0).sum()))

There are 0 rows with no missing values.


In [149]:
merged.tail()

,ACCIDENT_DATE,ACCIDENT_NO,ACCIDENT_TIME,ATMOSPH_COND,COMPLEX_INT_NO,COMPLEX_NODE,DAY_OF_WEEK,DEGREE_URBAN,HOUR,LAT,LIGHT_CONDITION,LON,NODE_ID,NODE_TYPE_INT,ROAD_GEOMETRY,SPEED_ZONE,SUBURB,near_id,ACCIDENT_DATE_ISO,LAST_7_DAYS,LAST_30_DAYS,LAST_365_DAYS,LAST_1825_DAYS,LAST_3650_DAYS,display_name,hwy_type,inter,intersection_segments,key,lanes,oneway,orig_id,osm_speed,segment_id,signal,streets,to,width,width_per_lane
171890,2018-12-10,T20180019126,08.45.00,1,0,0,5,MELB_URBAN,08,-37.81086,1,145.23477,337472,1,5,80,MAROONDAH,0046676,"(2018, 50, 1)",0,0,0,1,4,Ringwood Bypass,11,0.0,NaN,0,3,1,9964324,80,226515423-249447293-96366634,0,NaN,96366634,0,0
171894,2018-12-10,T20180019139,12.44.00,1,0,0,5,MELBOURNE_CBD,12,-37.81272,1,144.97216,335436,1,2,40,MELBOURNE,0013280,"(2018, 50, 1)",0,0,0,1,3,Little Collins Street between Spring Street an...,1,0.0,NaN,0,0,1,9918245,40,4306021-2279153639-589402289,0,NaN,589402289,0,0
171893,2018-12-10,T20180019137,10.00.00,1,0,0,6,MELB_URBAN,10,-37.77496,1,144.97114,35987,0,1,40,MORELAND,65293,"(2018, 50, 1)",0,0,0,7,9,NaN,6,NaN,4.0,0,0,0,99203901,50,22240454-32997268-97506078,0,NaN,97512044,0,0
171888,2018-12-10,T20180019119,03.50.00,1,0,0,5,MELB_URBAN,03,-37.65250,3,145.02324,335426,1,5,60,WHITTLESEA,89224,"(2018, 50, 1)",0,0,0,0,1,NaN,9,NaN,13.0,0,3,1,99228702,0,390157693-364653757-3373135393,0,NaN,364653759,0,0
171905,2018-12-10,T20180019164,15.15.00,1,9083,1,5,MELB_URBAN,15,-37.74359,1,145.04758,56886,0,2,60,BANYULE,73541,"(2018, 50, 1)",0,0,1,1,3,NaN,5,NaN,7.0,0,2,0,99127286,60,51970487-317317429-317316707,0,NaN,317316707,0,0


In [143]:
aggregated_df.shape

(279420, 15)

In [150]:
import numpy as np
import pandas as pd
import scipy.stats as ss
from sklearn.metrics import roc_auc_score
import os
import json
import argparse
import yaml
BASE_DIR = 'C:/Users/Daniel/Documents/ML/Transurban'

In [167]:
config_file= 'src/config/Melbourne.yml'
datadir = 'data/Melbourne'

In [168]:
with open(config_file) as f:
    config = yaml.safe_load(f)

In [169]:
config

{'city': 'Melbourne, Victoria, Aus',
 'city_latitude': 'None',
 'city_longitude': 'None',
 'timezone': 'Australia/Sydney',
 'name': 'Melbourne',
 'startdate': None,
 'enddate': None,
 'crash_files': 'C:\\Users\\Daniel\\Documents\\ML\\Transurban\\data\\Melbourne\\raw\\crash\\crash C:\\Users\\Daniel\\Documents\\ML\\Transurban\\data\\Melbourne\\raw\\crash\\map C:\\Users\\Daniel\\Documents\\ML\\Transurban\\data\\Melbourne\\raw\\crash\\map_inters C:\\Users\\Daniel\\Documents\\ML\\Transurban\\data\\Melbourne\\raw\\crash\\atmosphere',
 'cat_feat': ['HOUR',
  'DAY_OF_WEEK',
  'DEGREE_URBAN',
  'LIGHT_CONDITION',
  'ATMOSPH_COND',
  'NODE_TYPE_INT',
  'COMPLEX_INT_NO',
  'hwy_type',
  'inter',
  'intersection_segments',
  'key',
  'lanes',
  'oneway',
  'signal',
  'streets'],
 'cont_feat': ['SPEED_ZONE',
  'osm_speed',
  'LAST_7_DAYS',
  'LAST_30_DAYS',
  'LAST_365_DAYS',
  'LAST_1825_DAYS',
  'LAST_3650_DAYS']}

In [171]:
a = config['cat_feat']
type(a)

list

In [154]:
print('Reading in seg_data from config[seg_data] at: \n\n', config['seg_data'])
print('Config file looks like \n \n', config, '\n\n')

Reading in seg_data from config[seg_data] at: 

 crash_OSM_merged.csv.gz
Config file looks like 
 
 {'city': 'Melbourne, Victoria, Australia', 'city_latitude': 'None', 'city_longitude': 'None', 'timezone': 'Australia/Sydney', 'city_radius': 20, 'name': 'Melbourne', 'startdate': None, 'enddate': None, 'level': 'segment', 'crash_files': {'C:\\Users\\Daniel\\Documents\\ML\\Transurban\\data\\Melbourne\\raw\\crash\\crash': {'required': {'id': None, 'latitude': None, 'longitude': None, 'date_complete': None, 'date_year': None, 'date_month': None, 'date_day': None, 'time': None, 'time_format': None}}, 'C:\\Users\\Daniel\\Documents\\ML\\Transurban\\data\\Melbourne\\raw\\crash\\map': None, 'C:\\Users\\Daniel\\Documents\\ML\\Transurban\\data\\Melbourne\\raw\\crash\\map_inters': None, 'C:\\Users\\Daniel\\Documents\\ML\\Transurban\\data\\Melbourne\\raw\\crash\\atmosphere': None}, 'seg_data': 'crash_OSM_merged.csv.gz'} 




In [155]:
DATA_FP = os.path.join(BASE_DIR, 'data', config['name'])
PROCESSED_DATA_FP = os.path.join(BASE_DIR, 'data', config['name'], 'processed/')
seg_data = os.path.join(PROCESSED_DATA_FP, config['seg_data'])

In [157]:
data = pd.read_csv(seg_data, dtype={'segment_id': 'str'})

In [161]:
data.sort_values(['ACCIDENT_DATE', 'segment_id'], inplace=True)

data.head()

In [164]:
data.head()

,segment_id,ACCIDENT_DATE,ACCIDENT_NO,ACCIDENT_TIME,ATMOSPH_COND,COMPLEX_INT_NO,COMPLEX_NODE,DAY_OF_WEEK,DEGREE_URBAN,HOUR,LAT,LIGHT_CONDITION,LON,NODE_ID,NODE_TYPE_INT,ROAD_GEOMETRY,SPEED_ZONE,SUBURB,near_id,ACCIDENT_DATE_ISO,LAST_7_DAYS,LAST_30_DAYS,LAST_365_DAYS,LAST_1825_DAYS,LAST_3650_DAYS,display_name,hwy_type,inter,intersection_segments,key,lanes,oneway,orig_id,osm_speed,signal,streets,to,width,width_per_lane
26,131710567-248746942-576845616,2006-01-01,T20060031159,08.43.00,1,0,0,1,MELB_URBAN,8,-37.82294,1,144.98913,116554,1.0,5,60,MELBOURNE,17003,"(2005, 52, 7)",0,0,0,0,0,Punt Road from Tanner Street,11,0.0,0.0,0,3,1,9923439,60,0,0.0,576845616,0,0
33,179391104-301993232-2299164392,2006-01-01,T20060004649,16.09.00,2,0,0,1,MELB_URBAN,16,-37.71816,1,144.96265,25970,1.0,5,60,MORELAND,356,"(2005, 52, 7)",0,0,0,0,0,NaN,3,0.0,0.0,0,1,1,99534,60,0,0.0,2299164392,0,0
30,202258273-3091734981-301992581,2006-01-01,T20060000317,06.30.00,1,0,0,1,MELB_URBAN,6,-37.71253,1,144.96167,3079,1.0,5,70,MORELAND,298,"(2005, 52, 7)",0,0,0,0,0,Sydney Road (Hume Highway),5,0.0,0.0,0,3,1,99425,70,0,0.0,301992581,0,0
20,203417605-1288852225-33999469,2006-01-01,T20060025550,00.01.00,9,0,0,1,MELB_URBAN,0,-37.89321,3,145.31603,97834,1.0,5,70,YARRA RANGES,50337,"(2005, 52, 7)",0,0,0,0,0,Burwood Highway,2,0.0,0.0,0,2,1,9969084,70,0,0.0,33999469,0,0
38,24672962-268220094-268170836,2006-01-01,T20060004653,05.58.00,1,0,0,1,MELB_URBAN,5,-37.77066,2,144.79180,202058,1.0,5,80,BRIMBANK,165378,"(2005, 52, 7)",0,0,0,0,0,Ballarat Road between Western Highway Offramp ...,11,0.0,0.0,0,3,1,99226902,0,0,0.0,268170836,0,0


In [ ]:
if config['level'] == 'week':
    week = int(config['time_target'][0])
    year = int(config['time_target'][1])

    # get segments with non-zero crashes
    # this is necessary to constrain the problem for weekly predictions
    data = data.set_index('segment_id').loc[data.groupby('segment_id').crash.sum() > 0]
    data.reset_index(inplace=True)

f_cat, f_cont, features = get_features(config, data, PROCESSED_DATA_FP)

print('Have just read in f_cat, f_cont and features which respectively look g like \n', f_cat, '\n', f_cont, '\n', features)

# grab the highest values from each column
data_segs = data.groupby('segment_id')[f_cont + f_cat].max()
data_segs.reset_index(inplace=True)
data_segs = add_extra_features(data, data_segs, config, PROCESSED_DATA_FP)

print('Have just got the highest values from each column for each segment id, and then added extra features: \n\n', data_segs.head)

data_segs, features, lm_features = process_features(
    features, config, f_cat, f_cont, data_segs)

if config['level'] == 'week':
    # create lagged crash values
    crash_lags = format_crash_data(data, 'crash', week, year)

    # add to features
    crash_cols = ['pre_week', 'pre_month', 'pre_quarter', 'avg_week']
    features += crash_cols
    # create model data
    data_model = crash_lags.merge(data_segs, left_on='segment_id', right_on='segment_id')
else:
    # if not week, get any crash 0/1
    any_crash = data.groupby('segment_id')['crash'].max()
    any_crash = (any_crash > 0).astype(int)
    any_crash.name = 'target'
    data_model = data_segs.set_index('segment_id').join(any_crash).reset_index()
print("full features:{}".format(features))

print('\n\n Data_model: \n\n', data_model)
print('\n\n features:', features)
print('\n\n lm_features:', lm_features)
print('\n\n config[level]:', config['level'])
print('\n\n Process_data_fp:', PROCESSED_DATA_FP)
# data_model.to_csv('myDataModel.csv')

initialize_and_run(data_model, features, lm_features, config['level'], PROCESSED_DATA_FP)

In [174]:
temp = pd.get_dummies(data['hwy_type'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,29,30,31,32,33,35,36,37,38,42,43
26,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
39,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [175]:
f = 'hwy_type'
temp = pd.get_dummies(data['hwy_type'])
temp.columns = [f + str(c) for c in temp.columns]


In [176]:
temp

,hwy_type0,hwy_type1,hwy_type2,hwy_type3,hwy_type4,hwy_type5,hwy_type6,hwy_type7,hwy_type8,hwy_type9,hwy_type10,hwy_type11,hwy_type12,hwy_type13,hwy_type14,hwy_type15,hwy_type16,hwy_type17,hwy_type18,hwy_type19,hwy_type21,hwy_type22,hwy_type23,hwy_type24,hwy_type25,hwy_type26,hwy_type27,hwy_type29,hwy_type30,hwy_type31,hwy_type32,hwy_type33,hwy_type35,hwy_type36,hwy_type37,hwy_type38,hwy_type42,hwy_type43
26,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
39,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [177]:
datacopy = data.copy()


In [178]:
datacopy = pd.concat([datacopy, temp], axis=1)

In [179]:
datacopy

,segment_id,ACCIDENT_DATE,ACCIDENT_NO,ACCIDENT_TIME,ATMOSPH_COND,COMPLEX_INT_NO,COMPLEX_NODE,DAY_OF_WEEK,DEGREE_URBAN,HOUR,LAT,LIGHT_CONDITION,LON,NODE_ID,NODE_TYPE_INT,ROAD_GEOMETRY,SPEED_ZONE,SUBURB,near_id,ACCIDENT_DATE_ISO,LAST_7_DAYS,LAST_30_DAYS,LAST_365_DAYS,LAST_1825_DAYS,LAST_3650_DAYS,display_name,hwy_type,inter,intersection_segments,key,lanes,oneway,orig_id,osm_speed,signal,streets,to,width,width_per_lane,hwy_type0,hwy_type1,hwy_type2,hwy_type3,hwy_type4,hwy_type5,hwy_type6,hwy_type7,hwy_type8,hwy_type9,hwy_type10,hwy_type11,hwy_type12,hwy_type13,hwy_type14,hwy_type15,hwy_type16,hwy_type17,hwy_type18,hwy_type19,hwy_type21,hwy_type22,hwy_type23,hwy_type24,hwy_type25,hwy_type26,hwy_type27,hwy_type29,hwy_type30,hwy_type31,hwy_type32,hwy_type33,hwy_type35,hwy_type36,hwy_type37,hwy_type38,hwy_type42,hwy_type43
26,131710567-248746942-576845616,2006-01-01,T20060031159,08.43.00,1,0,0,1,MELB_URBAN,8,-37.82294,1,144.98913,116554,1.0,5,60,MELBOURNE,17003,"(2005, 52, 7)",0,0,0,0,0,Punt Road from Tanner Street,11,0.0,0.0,0,3,1,9923439,60,0,0.0,576845616,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33,179391104-301993232-2299164392,2006-01-01,T20060004649,16.09.00,2,0,0,1,MELB_URBAN,16,-37.71816,1,144.96265,25970,1.0,5,60,MORELAND,356,"(2005, 52, 7)",0,0,0,0,0,NaN,3,0.0,0.0,0,1,1,99534,60,0,0.0,2299164392,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30,202258273-3091734981-301992581,2006-01-01,T20060000317,06.30.00,1,0,0,1,MELB_URBAN,6,-37.71253,1,144.96167,3079,1.0,5,70,MORELAND,298,"(2005, 52, 7)",0,0,0,0,0,Sydney Road (Hume Highway),5,0.0,0.0,0,3,1,99425,70,0,0.0,301992581,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20,203417605-1288852225-33999469,2006-01-01,T20060025550,00.01.00,9,0,0,1,MELB_URBAN,0,-37.89321,3,145.31603,97834,1.0,5,70,YARRA RANGES,50337,"(2005, 52, 7)",0,0,0,0,0,Burwood Highway,2,0.0,0.0,0,2,1,9969084,70,0,0.0,33999469,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
38,24672962-268220094-268170836,2006-01-01,T20060004653,05.58.00,1,0,0,1,MELB_URBAN,5,-37.77066,2,144.79180,202058,1.0,5,80,BRIMBANK,165378,"(2005, 52, 7)",0,0,0,0,0,Ballarat Road between Western Highway Offramp ...,11,0.0,0.0,0,3,1,99226902,0,0,0.0,268170836,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,24673004-268171091-268171097,2006-01-01,T20060008186,14.50.00,2,0,0,1,MELB_URBAN,14,-37.77085,1,144.79004,54677,0.0,2,80,BRIMBANK,66569,"(2005, 52, 7)",0,0,0,0,0,0,11,0.0,13.0,0,3,1,99226953,0,0,0.0,36713344,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,26489472-290334331-290334318,2006-01-01,T20060005894,01.20.00,1,0,0,1,MELB_URBAN,1,-37.79265,3,144.84699,202066,1.0,5,50,MARIBYRNONG,101669,"(2005, 52, 7)",0,0,0,0,0,Pritchard Avenue between Kent Street and Howel...,0,0.0,0.0,0,0,0,99139045,50,0,0.0,290334318,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
39,265722452-1833868342-1833868341,2006-01-01,T20060002275,00.05.00,1,0,0,1,RURAL_VICTORIA,0,-37.58527,3,144.71446,202189,1.0,5,60,HUME,104546,"(2005, 52, 7)",0,0,0,0,0,Mitchells Lane between Plante Court and Lawson...,6,0.0,0.0,0,0,0,99143203,0,0,0.0,1833868341,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,295188288-155649178-295849746,2006-01-01,T20060011459,13.10.00,1,0,0,1,MELB_URBAN,13,-37.81530,1,145.26929,55650,0.0,2,70,MAROONDAH,54917,"(2005, 52, 7)",0,0,0,0,0,0,5,0.0,7.0,0,2,0,9971304,70,0,0.0,310953723,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37,31227479-347256611-347256591,2006-01-01,T20060004668,08.55.00,1,0,0,1,MELB_URBAN,8,-37.86741,1,144.68579,202054,1.0,5,50,WYNDHAM,26701,"(2005, 52, 7)",0,0,0,0,0,Bartlett Crescent between Symons Avenue and Po...,0,0.0,0.0,0,0,0,9936654,0,0,0.0,347256591,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [181]:
features = ['hwy_type']

In [185]:
f_cont = ['osm_speed', 'LAST_3650_DAYS']

In [187]:
np.log(datacopy['osm_speed'] + 1)

26        4.110874
33        4.110874
30        4.262680
20        4.262680
38        0.000000
13        0.000000
8         3.931826
39        0.000000
0         4.262680
37        0.000000
4         3.931826
29        4.110874
1         4.262680
15        4.615121
6         3.713572
10        4.394449
34        4.110874
28        0.000000
24        0.000000
3         0.000000
27        4.110874
36        4.110874
19        0.000000
2         0.000000
25        4.110874
12        4.110874
11        4.615121
16        4.262680
32        4.615121
7         4.615121
            ...   
123221    4.394449
123211    4.110874
123217    4.262680
123219    4.394449
123222    4.110874
123214    0.000000
123216    4.110874
123220    4.615121
123210    4.394449
123209    4.615121
123212    0.000000
123213    4.394449
123218    4.262680
123226    0.000000
123237    3.931826
123235    4.394449
123224    3.931826
123229    4.110874
123232    4.615121
123238    0.000000
123231    4.262680
123223    4.

In [183]:
for f in f_cont:
    datacopy['log_%s' % f] = np.log(datacopy[f] + 1)
    features += ['log_%s' % f]
    lm_features += ['log_%s' % f]

['hwy_type',
 'hwy_type0',
 'hwy_type1',
 'hwy_type2',
 'hwy_type3',
 'hwy_type4',
 'hwy_type5',
 'hwy_type6',
 'hwy_type7',
 'hwy_type8',
 'hwy_type9',
 'hwy_type10',
 'hwy_type11',
 'hwy_type12',
 'hwy_type13',
 'hwy_type14',
 'hwy_type15',
 'hwy_type16',
 'hwy_type17',
 'hwy_type18',
 'hwy_type19',
 'hwy_type21',
 'hwy_type22',
 'hwy_type23',
 'hwy_type24',
 'hwy_type25',
 'hwy_type26',
 'hwy_type27',
 'hwy_type29',
 'hwy_type30',
 'hwy_type31',
 'hwy_type32',
 'hwy_type33',
 'hwy_type35',
 'hwy_type36',
 'hwy_type37',
 'hwy_type38',
 'hwy_type42',
 'hwy_type43']

In [188]:
a = 'log_%s' % features

In [189]:
a

"log_['hwy_type', 'hwy_type0', 'hwy_type1', 'hwy_type2', 'hwy_type3', 'hwy_type4', 'hwy_type5', 'hwy_type6', 'hwy_type7', 'hwy_type8', 'hwy_type9', 'hwy_type10', 'hwy_type11', 'hwy_type12', 'hwy_type13', 'hwy_type14', 'hwy_type15', 'hwy_type16', 'hwy_type17', 'hwy_type18', 'hwy_type19', 'hwy_type21', 'hwy_type22', 'hwy_type23', 'hwy_type24', 'hwy_type25', 'hwy_type26', 'hwy_type27', 'hwy_type29', 'hwy_type30', 'hwy_type31', 'hwy_type32', 'hwy_type33', 'hwy_type35', 'hwy_type36', 'hwy_type37', 'hwy_type38', 'hwy_type42', 'hwy_type43']"

In [201]:
datacopy.loc[datacopy.segment_id.map(lambda x: x[:2] == '00')]

,segment_id,ACCIDENT_DATE,ACCIDENT_NO,ACCIDENT_TIME,ATMOSPH_COND,COMPLEX_INT_NO,COMPLEX_NODE,DAY_OF_WEEK,DEGREE_URBAN,HOUR,LAT,LIGHT_CONDITION,LON,NODE_ID,NODE_TYPE_INT,ROAD_GEOMETRY,SPEED_ZONE,SUBURB,near_id,ACCIDENT_DATE_ISO,LAST_7_DAYS,LAST_30_DAYS,LAST_365_DAYS,LAST_1825_DAYS,LAST_3650_DAYS,display_name,hwy_type,inter,intersection_segments,key,lanes,oneway,orig_id,osm_speed,signal,streets,to,width,width_per_lane,hwy_type0,hwy_type1,hwy_type2,hwy_type3,hwy_type4,hwy_type5,hwy_type6,hwy_type7,hwy_type8,hwy_type9,hwy_type10,hwy_type11,hwy_type12,hwy_type13,hwy_type14,hwy_type15,hwy_type16,hwy_type17,hwy_type18,hwy_type19,hwy_type21,hwy_type22,hwy_type23,hwy_type24,hwy_type25,hwy_type26,hwy_type27,hwy_type29,hwy_type30,hwy_type31,hwy_type32,hwy_type33,hwy_type35,hwy_type36,hwy_type37,hwy_type38,hwy_type42,hwy_type43


In [207]:
f = ['a', 'b', 'c', 'a']
g = ['a']
h = ['b']

In [208]:
set(f)

{'a', 'b', 'c'}

In [209]:
set(f) - set(g + h)

{'c'}

In [210]:
set(g + h)

{'a', 'b'}

In [211]:
data['target'] = 1

In [216]:
features

['hwy_type',
 'hwy_type0',
 'hwy_type1',
 'hwy_type2',
 'hwy_type3',
 'hwy_type4',
 'hwy_type5',
 'hwy_type6',
 'hwy_type7',
 'hwy_type8',
 'hwy_type9',
 'hwy_type10',
 'hwy_type11',
 'hwy_type12',
 'hwy_type13',
 'hwy_type14',
 'hwy_type15',
 'hwy_type16',
 'hwy_type17',
 'hwy_type18',
 'hwy_type19',
 'hwy_type21',
 'hwy_type22',
 'hwy_type23',
 'hwy_type24',
 'hwy_type25',
 'hwy_type26',
 'hwy_type27',
 'hwy_type29',
 'hwy_type30',
 'hwy_type31',
 'hwy_type32',
 'hwy_type33',
 'hwy_type35',
 'hwy_type36',
 'hwy_type37',
 'hwy_type38',
 'hwy_type42',
 'hwy_type43']

In [221]:
other_features = ['osm_speed', 'ATMOSPH_COND']

In [223]:
datacopy[features + other_features].head()

,hwy_type,hwy_type0,hwy_type1,hwy_type2,hwy_type3,hwy_type4,hwy_type5,hwy_type6,hwy_type7,hwy_type8,hwy_type9,hwy_type10,hwy_type11,hwy_type12,hwy_type13,hwy_type14,hwy_type15,hwy_type16,hwy_type17,hwy_type18,hwy_type19,hwy_type21,hwy_type22,hwy_type23,hwy_type24,hwy_type25,hwy_type26,hwy_type27,hwy_type29,hwy_type30,hwy_type31,hwy_type32,hwy_type33,hwy_type35,hwy_type36,hwy_type37,hwy_type38,hwy_type42,hwy_type43,osm_speed,ATMOSPH_COND
26,11,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,60,1
33,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,60,2
30,5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,70,1
20,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,70,9
38,11,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [220]:
datacopy.head()

,segment_id,ACCIDENT_DATE,ACCIDENT_NO,ACCIDENT_TIME,ATMOSPH_COND,COMPLEX_INT_NO,COMPLEX_NODE,DAY_OF_WEEK,DEGREE_URBAN,HOUR,LAT,LIGHT_CONDITION,LON,NODE_ID,NODE_TYPE_INT,ROAD_GEOMETRY,SPEED_ZONE,SUBURB,near_id,ACCIDENT_DATE_ISO,LAST_7_DAYS,LAST_30_DAYS,LAST_365_DAYS,LAST_1825_DAYS,LAST_3650_DAYS,display_name,hwy_type,inter,intersection_segments,key,lanes,oneway,orig_id,osm_speed,signal,streets,to,width,width_per_lane,hwy_type0,hwy_type1,hwy_type2,hwy_type3,hwy_type4,hwy_type5,hwy_type6,hwy_type7,hwy_type8,hwy_type9,hwy_type10,hwy_type11,hwy_type12,hwy_type13,hwy_type14,hwy_type15,hwy_type16,hwy_type17,hwy_type18,hwy_type19,hwy_type21,hwy_type22,hwy_type23,hwy_type24,hwy_type25,hwy_type26,hwy_type27,hwy_type29,hwy_type30,hwy_type31,hwy_type32,hwy_type33,hwy_type35,hwy_type36,hwy_type37,hwy_type38,hwy_type42,hwy_type43
26,131710567-248746942-576845616,2006-01-01,T20060031159,08.43.00,1,0,0,1,MELB_URBAN,8,-37.82294,1,144.98913,116554,1.0,5,60,MELBOURNE,17003,"(2005, 52, 7)",0,0,0,0,0,Punt Road from Tanner Street,11,0.0,0.0,0,3,1,9923439,60,0,0.0,576845616,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33,179391104-301993232-2299164392,2006-01-01,T20060004649,16.09.00,2,0,0,1,MELB_URBAN,16,-37.71816,1,144.96265,25970,1.0,5,60,MORELAND,356,"(2005, 52, 7)",0,0,0,0,0,NaN,3,0.0,0.0,0,1,1,99534,60,0,0.0,2299164392,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30,202258273-3091734981-301992581,2006-01-01,T20060000317,06.30.00,1,0,0,1,MELB_URBAN,6,-37.71253,1,144.96167,3079,1.0,5,70,MORELAND,298,"(2005, 52, 7)",0,0,0,0,0,Sydney Road (Hume Highway),5,0.0,0.0,0,3,1,99425,70,0,0.0,301992581,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20,203417605-1288852225-33999469,2006-01-01,T20060025550,00.01.00,9,0,0,1,MELB_URBAN,0,-37.89321,3,145.31603,97834,1.0,5,70,YARRA RANGES,50337,"(2005, 52, 7)",0,0,0,0,0,Burwood Highway,2,0.0,0.0,0,2,1,9969084,70,0,0.0,33999469,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
38,24672962-268220094-268170836,2006-01-01,T20060004653,05.58.00,1,0,0,1,MELB_URBAN,5,-37.77066,2,144.79180,202058,1.0,5,80,BRIMBANK,165378,"(2005, 52, 7)",0,0,0,0,0,Ballarat Road between Western Highway Offramp ...,11,0.0,0.0,0,3,1,99226902,0,0,0.0,268170836,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [225]:
config_file = 'src/config/Melbourne.yml'
datadir = 'data/Melbourne'

In [226]:
with open(config_file) as f:
    config = yaml.safe_load(f)

# Create required data paths
DATA_FP = os.path.join(BASE_DIR, 'data', config['name'])
PROCESSED_DATA_FP = os.path.join(BASE_DIR, 'data', config['name'], 'processed/')
merged_data_path = os.path.join(PROCESSED_DATA_FP, config['merged_data'])
print(('Outputting to: %s' % PROCESSED_DATA_FP))

Outputting to: C:/Users/Daniel/Documents/ML/Transurban\data\Melbourne\processed/


In [230]:
def get_features(config, data, datadir):
    """
    Get features from the feature list created during data generation / or the list specified during init_city.
    """

    cont_feat = config['cont_feat']
    cat_feat = config['cat_feat']

    # Dropping continuous features that don't exist
    cont_feat_found = []
    for f in cont_feat:
        if f not in data.columns.values:
            print("Feature " + f + " not found, skipping")
        else:
            cont_feat_found.append(f)

    # Dropping categorical features that don't exist
    cat_feat_found = []
    for f in cat_feat:
        if f not in data.columns.values:
            print("Feature " + f + " not found, skipping")
        else:
            cat_feat_found.append(f)

    # Create featureset holder
    features = cont_feat_found + cat_feat_found

    return cont_feat_found, cat_feat_found, features

In [237]:
def process_features(data, features, config, f_cat, f_cont):

    print('Within train_model.process_features')

    # Features for linear model
    linear_model_features = features

    # Turn categorical variables into one-hot representation through get_dummies
    # Append the newly named one-hot variables [e.g. hwy_type23] to our data frame
    # Append the new feature names to our feature list
    # For linear model features leave out the first one [e.g. hwy_type0 for intercept (?)]
    print('Processing categorical variables [one-hot encoding]')
    for f in f_cat:
        temp = pd.get_dummies(data[f])
        temp.columns = [f + '_' + str(c) for c in temp.columns]
        data = pd.concat([data, temp], axis=1)
        features += temp.columns.tolist()
        linear_model_features += temp.columns.tolist()[1:]

    # Turn continuous variables into their log [add one to avoid -inf errors]
    # Append new feature name to relevant lists[e.g. log_width]
    print('Processing continuous variables [log-transform]')
    for f in f_cont:
        data['log_%s' % f] = np.log(data[f] + 1)
        features += ['log_%s' % f]
        linear_model_features += ['log_%s' % f]

    # Create a new feature 'intersection'
    # If a segment_id does not start with '00', pass it a 1 to intersection
    # Otherwise, pass it a 0 to intersection
    data['intersection'] = data.segment_id.map(lambda x: x[:2] != '00').astype(int)
    features += ['intersection']
    linear_model_features += ['intersection']

    # Remove duplicated features
    # e.g. if features = ['a', 'b', 'c', 'b'], f_cat = ['a'] and f_cont=['b']
    # then set(features) = {'a', 'b', 'c'} and set(f_cat + f_cont) = {'a', 'b'}

    features = list(set(features) - set(f_cat + f_cont))
    linear_model_features = list(set(linear_model_features) - set(f_cat + f_cont))

    return data, features, linear_model_features

In [238]:
# Read in data
data = pd.read_csv(merged_data_path, dtype={'segment_id': 'str'})
data.sort_values(['ACCIDENT_DATE', 'segment_id'], inplace=True)

# Get all features that exist within dataset and are being used
f_cont, f_cat, features = get_features(config, data, PROCESSED_DATA_FP)
print('Our categorical features are:', f_cat)
print('Our continuous features are:', f_cont)

Our categorical features are: ['HOUR', 'DAY_OF_WEEK', 'DEGREE_URBAN', 'LIGHT_CONDITION', 'ATMOSPH_COND', 'NODE_TYPE_INT', 'COMPLEX_INT_NO', 'hwy_type', 'inter', 'intersection_segments', 'key', 'lanes', 'oneway', 'signal', 'streets']
Our continuous features are: ['SPEED_ZONE', 'osm_speed', 'LAST_7_DAYS', 'LAST_30_DAYS', 'LAST_365_DAYS', 'LAST_1825_DAYS', 'LAST_3650_DAYS']


In [239]:
data, features, linear_model_features = process_features(data, features, config, f_cat, f_cont)

Within train_model.process_features
Processing categorical variables [one-hot encoding]
Processing continuous variables [log-transform]


In [240]:
data

,segment_id,ACCIDENT_DATE,ACCIDENT_NO,ACCIDENT_TIME,ATMOSPH_COND,COMPLEX_INT_NO,COMPLEX_NODE,DAY_OF_WEEK,DEGREE_URBAN,HOUR,LAT,LIGHT_CONDITION,LON,NODE_ID,NODE_TYPE_INT,ROAD_GEOMETRY,SPEED_ZONE,SUBURB,near_id,ACCIDENT_DATE_ISO,LAST_7_DAYS,LAST_30_DAYS,LAST_365_DAYS,LAST_1825_DAYS,LAST_3650_DAYS,display_name,hwy_type,inter,intersection_segments,key,lanes,oneway,orig_id,osm_speed,signal,streets,to,width,width_per_lane,HOUR_0,HOUR_1,HOUR_2,HOUR_3,HOUR_4,HOUR_5,HOUR_6,HOUR_7,HOUR_8,HOUR_9,HOUR_10,HOUR_11,HOUR_12,HOUR_13,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23,DAY_OF_WEEK_0,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6,DAY_OF_WEEK_7,DEGREE_URBAN_MELBOURNE_CBD,DEGREE_URBAN_MELB_URBAN,DEGREE_URBAN_RURAL_VICTORIA,DEGREE_URBAN_SMALL_CITIES,DEGREE_URBAN_SMALL_TOWNS,DEGREE_URBAN_TOWNS,LIGHT_CONDITION_1,LIGHT_CONDITION_2,LIGHT_CONDITION_3,LIGHT_CONDITION_4,LIGHT_CONDITION_5,LIGHT_CONDITION_6,LIGHT_CONDITION_9,ATMOSPH_COND_1,ATMOSPH_COND_2,ATMOSPH_COND_4,ATMOSPH_COND_5,ATMOSPH_COND_6,ATMOSPH_COND_7,ATMOSPH_COND_9,NODE_TYPE_INT_0.0,NODE_TYPE_INT_1.0,NODE_TYPE_INT_2.0,COMPLEX_INT_NO_0,COMPLEX_INT_NO_1000,COMPLEX_INT_NO_1001,COMPLEX_INT_NO_1002,COMPLEX_INT_NO_1004,COMPLEX_INT_NO_1005,COMPLEX_INT_NO_1006,COMPLEX_INT_NO_1013,COMPLEX_INT_NO_1014,COMPLEX_INT_NO_1015,COMPLEX_INT_NO_1016,COMPLEX_INT_NO_1017,COMPLEX_INT_NO_1018,COMPLEX_INT_NO_1019,COMPLEX_INT_NO_1020,COMPLEX_INT_NO_1021,COMPLEX_INT_NO_1022,COMPLEX_INT_NO_5003,COMPLEX_INT_NO_5006,COMPLEX_INT_NO_5014,COMPLEX_INT_NO_5019,COMPLEX_INT_NO_5020,COMPLEX_INT_NO_5025,COMPLEX_INT_NO_5027,COMPLEX_INT_NO_5028,COMPLEX_INT_NO_5031,COMPLEX_INT_NO_5035,COMPLEX_INT_NO_5036,COMPLEX_INT_NO_5037,COMPLEX_INT_NO_5038,COMPLEX_INT_NO_5039,COMPLEX_INT_NO_5042,COMPLEX_INT_NO_5043,COMPLEX_INT_NO_5045,COMPLEX_INT_NO_5047,COMPLEX_INT_NO_5048,COMPLEX_INT_NO_5050,COMPLEX_INT_NO_5051,COMPLEX_INT_NO_5054,COMPLEX_INT_NO_5055,COMPLEX_INT_NO_5057,COMPLEX_INT_NO_5058,COMPLEX_INT_NO_5061,COMPLEX_INT_NO_5062,COMPLEX_INT_NO_5063,COMPLEX_INT_NO_5064,COMPLEX_INT_NO_5065,COMPLEX_INT_NO_5066,COMPLEX_INT_NO_5069,COMPLEX_INT_NO_5070,COMPLEX_INT_NO_5071,COMPLEX_INT_NO_5072,COMPLEX_INT_NO_5073,COMPLEX_INT_NO_5074,COMPLEX_INT_NO_5075,COMPLEX_INT_NO_5077,COMPLEX_INT_NO_5078,COMPLEX_INT_NO_5079,COMPLEX_INT_NO_5080,COMPLEX_INT_NO_5081,COMPLEX_INT_NO_5082,COMPLEX_INT_NO_5086,COMPLEX_INT_NO_5087,COMPLEX_INT_NO_5088,COMPLEX_INT_NO_5089,COMPLEX_INT_NO_5090,COMPLEX_INT_NO_5092,COMPLEX_INT_NO_5095,COMPLEX_INT_NO_5097,COMPLEX_INT_NO_5098,COMPLEX_INT_NO_5099,COMPLEX_INT_NO_5101,COMPLEX_INT_NO_5105,COMPLEX_INT_NO_5106,COMPLEX_INT_NO_5110,COMPLEX_INT_NO_5112,COMPLEX_INT_NO_5113,COMPLEX_INT_NO_5114,COMPLEX_INT_NO_5116,COMPLEX_INT_NO_5120,COMPLEX_INT_NO_5121,COMPLEX_INT_NO_5122,COMPLEX_INT_NO_5123,COMPLEX_INT_NO_5124,COMPLEX_INT_NO_5127,COMPLEX_INT_NO_5129,COMPLEX_INT_NO_5131,COMPLEX_INT_NO_5135,COMPLEX_INT_NO_5137,COMPLEX_INT_NO_5140,COMPLEX_INT_NO_5144,COMPLEX_INT_NO_5145,COMPLEX_INT_NO_5146,COMPLEX_INT_NO_5147,COMPLEX_INT_NO_5148,COMPLEX_INT_NO_5149,COMPLEX_INT_NO_5150,COMPLEX_INT_NO_5151,COMPLEX_INT_NO_5153,COMPLEX_INT_NO_5155,COMPLEX_INT_NO_5156,COMPLEX_INT_NO_5157,COMPLEX_INT_NO_5158,COMPLEX_INT_NO_5161,COMPLEX_INT_NO_5163,COMPLEX_INT_NO_5164,COMPLEX_INT_NO_5165,COMPLEX_INT_NO_5166,COMPLEX_INT_NO_5167,COMPLEX_INT_NO_5168,COMPLEX_INT_NO_5169,COMPLEX_INT_NO_5170,COMPLEX_INT_NO_5171,COMPLEX_INT_NO_5172,COMPLEX_INT_NO_5175,COMPLEX_INT_NO_5176,COMPLEX_INT_NO_5177,COMPLEX_INT_NO_5180,COMPLEX_INT_NO_5182,COMPLEX_INT_NO_5183,COMPLEX_INT_NO_5186,COMPLEX_INT_NO_5187,COMPLEX_INT_NO_5188,COMPLEX_INT_NO_5189,COMPLEX_INT_NO_5190,COMPLEX_INT_NO_5195,COMPLEX_INT_NO_5196,COMPLEX_INT_NO_5198,COMPLEX_INT_NO_5199,COMPLEX_INT_NO_5200,COMPLEX_INT_NO_5202,COMPLEX_INT_NO_5204,COMPLEX_INT_NO_5205,COMPLEX_INT_NO_5206,COMPLEX_INT_NO_5208,COMPLEX_INT_NO_5209,COMPLEX_INT_NO_5211,COMPLEX_INT_NO_5212,COMPLEX_INT_NO_5215,COMPLEX_INT_NO_5216,COMPLEX_INT_NO_5218,COMPLEX_INT_NO_5219,COMPLEX_INT_NO_

In [300]:
max_date = data.ACCIDENT_DATE.max()
min_date = data.ACCIDENT_DATE.min()

In [301]:
max_date

'2018-12-10'

In [302]:
min_date

'2006-01-01'

In [311]:
from datetime import datetime

max_date_obj = datetime.strptime(max_date, '%Y-%m-%d')
min_date_obj = datetime.strptime(min_date, '%Y-%m-%d')
days_difference = (max_date_obj - min_date_obj).days

num_pos_data = len(data)
num_neg_data = num_pos_data
num_neg_data_day = int(num_neg_data / days_difference)

dates_in_range = pd.date_range(min_date, max_date).tolist()
dates_in_range_multiplied = dates_in_range * num_neg_data_day

hours = range()

In [312]:
my_dates

[Timestamp('2006-01-01 00:00:00', freq='D'),
 Timestamp('2006-01-02 00:00:00', freq='D'),
 Timestamp('2006-01-03 00:00:00', freq='D'),
 Timestamp('2006-01-04 00:00:00', freq='D'),
 Timestamp('2006-01-05 00:00:00', freq='D'),
 Timestamp('2006-01-06 00:00:00', freq='D'),
 Timestamp('2006-01-07 00:00:00', freq='D'),
 Timestamp('2006-01-08 00:00:00', freq='D'),
 Timestamp('2006-01-09 00:00:00', freq='D'),
 Timestamp('2006-01-10 00:00:00', freq='D'),
 Timestamp('2006-01-11 00:00:00', freq='D'),
 Timestamp('2006-01-12 00:00:00', freq='D'),
 Timestamp('2006-01-13 00:00:00', freq='D'),
 Timestamp('2006-01-14 00:00:00', freq='D'),
 Timestamp('2006-01-15 00:00:00', freq='D'),
 Timestamp('2006-01-16 00:00:00', freq='D'),
 Timestamp('2006-01-17 00:00:00', freq='D'),
 Timestamp('2006-01-18 00:00:00', freq='D'),
 Timestamp('2006-01-19 00:00:00', freq='D'),
 Timestamp('2006-01-20 00:00:00', freq='D'),
 Timestamp('2006-01-21 00:00:00', freq='D'),
 Timestamp('2006-01-22 00:00:00', freq='D'),
 Timestamp